# Capstone Project - The Battle of the Neighborhoods (Week 2)

## Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## 1. Introduction : Business Problem <a name="introduction"></a>

This project will try to find an **optimal location for a coffee shop**. Specifically, it will be targeted to **stakeholders interested in opening a coffee shop in Toronto City**.

Since there are already a lot of coffee shops in Toronto City, we will try to **find areas that still have a few of coffee shops**, and would also prefer that the locations **close to universities/ learning centres** as we target students as our main customer segment. We will look what kind of coffee shops are available near the area, and gain insight to decide what kind of coffee shops that we need to build, to differentiate us with the other existing coffee shops or/ and to understand the customer's preference on coffee.

We will use data science algorithm to explore and analyze the data to find the best neighbourhoods, and use the insights that we gain to decide where are the most optimals options and generate the final location chosen by the stakeholder.

## 2. Data <a name="data"></a>

Based on the business problem description, the **data that needed to be explored and analyzed** are:

* number of existing universities/ learning centres near the neighbourhood - our potential customers are students, we prefer the location that is reachable for the students
* number of existing coffee shops near the universities, the level of competitiveness in opening a new coffee shop in that area
* neighbourhood clustering to find good benchmark for the new coffee shop

The **data sources** that will be needed to extract/generate the required information are as follow:

* Neighbourhood, universities/ learning centres in Toronto City, and its latitude & longitude will be excerpt from Toronto Open Data (https://open.toronto.ca/dataset/wellbeing-youth-alternative-adult-education/), use K-means clustering algorithm to cluster the neighbourhood
* The existing coffee shops near the universities/ learning centres will be obtained using Foursquare API

## 3. Methodology <a name="methodology"></a>

As a database, I used **Toronto Open Data repository** (https://open.toronto.ca/dataset/wellbeing-youth-alternative-adult-education/) csv file in my analysis. The master data has the information on alternative schools and learning centres, also includes school boards and colleges and universities in Toronto City. We need this data since we target students as the main segment of customers of our new coffee shop.

The methodology will take steps as follows :<br>
<ul><li>We will <strong>detect areas of Toronto City that has universities and/or learning centres that have low coffee shop density </strong>. We will limit the analysis to area ~1km around the universities/ learning centres.</li>

<li>In first step we have collected the required data of learning centres & universities available in Toronto City : name, address, neighbourhood, municipality and longlat, from Toronto Open Data Repository. <strong>We need to clean and prepare the data first before we do the data exploratory.</strong></li>

<li>Second step in our analysis will be <strong>calculation and exploration of 'coffee shop density' across different universities/ learning centres neighbourhood</strong> - we will use <strong>Foursquare API to identify a few promising areas with high number of universities/ learning centres and low number of coffee shops (low coffee shop density)</strong> and then focus our attention on the most potential area. 
    
<li>In third and final step we will explore the related borough. We will use <strong>K-Means Clustering to identify general zones / common places neighborhoods</strong>.</li>

<li>In the discussion with stakeholders: we will <strong>take into consideration the cluster that is still underserved in radius of 1000 meters, we want the location to have universities/ learning centres, and in a busy neighburhood/ borough (common places such as : office, shopping mall, public transports, etc).</strong></li></ul>  

## 4. Analysis <a name="analysis"></a>

### Preparing & Constructing Dataframe

In [1]:
#import libaries
import pandas as pd
import numpy as np

In [2]:
# Read the online file by the URL
import pandas as pd
import numpy as np
file_path ="https://ckan0.cf.opendata.inter.prod-toronto.ca/download_resource/a8df4608-31a0-4206-950f-e9265ca1037c?format=csv&projection=4326"
df = pd.read_csv(file_path)
df.head(5)

,_id,OBJECTID,AGENCY_NAME,ORGANIZATION_ADDRESS,NEIGHBOURHOOD,OFFICE_PHONE,EMAIL,WEBSITE,ELIGIBILITY,DESCRIPTION_SERVICE,...,DATE_UPDATED,ADDRESS_POINT_ID,X,Y,LONGITUDE,LATITUDE,ADDRESS_FULL,MUNICIPALITY,POSTAL_CODE,geometry
0,1,262,Joint Training and Apprenticeship Committee,"936 Warden Ave, Toronto, ON M1L 4C9","Wexford-Maryvale, 119",416-759-9351,jtacoffice@ualocal46.org,"<a href=""http://www.ualocal46.org"" target=""_bl...",United Association Local 46 member * or enroll...,Apprenticeship programs in plumbing and steamf...,...,2015-07-22T04:00:00,395765,NaN,NaN,NaN,NaN,936 Warden Ave,Scarborough,M1L 4C9,"{u'type': u'Point', u'coordinates': (-79.29001..."
1,2,263,Ontario Industrial and Finishing Skills Centre,"130 Toro Rd, Unit C, Toronto, ON M3J 3M9","York University Heights, 27",416-635-7300,info@oifsc.com,"<a href=""http://www.oifsc.com"" target=""_blank""...","Minimum Grade 10 English, math and science or ...",Apprenticeship programs for painter decorators...,...,2015-07-27T04:00:00,9171851,NaN,NaN,NaN,NaN,130 Toro Rd,North York,M3J 3M9,"{u'type': u'Point', u'coordinates': (-79.48321..."
2,3,264,Toronto Ironworkers Local 721,"909 Kipling Ave, Toronto, ON M8Z 5H3","Islington-City Centre West, 14",416-232-1046,ironworkerslocal721@rogers.com,"<a href=""http://www.iw721.org"" target=""_blank""...",Must have Grade 10,Apprenticeship programs for ironworkers and ro...,...,2015-08-25T04:00:00,1011523,NaN,NaN,NaN,NaN,909 Kipling Ave,Etobicoke,M8Z 5H3,"{u'type': u'Point', u'coordinates': (-79.53300..."
3,4,265,Toronto Catholic District School Board,"Catholic Education Centre, 80 Sheppard Ave E, ...","Willowdale East, 51",416-222-8282 ext 5314 ; Admissions 416-222-828...,webmaster@tcdsb.org,"<a href=""http://www.tcdsb.org"" target=""_blank""...",Elementary level -- Roman Catholic children an...,Full academic program in a Roman Catholic envi...,...,2015-07-08T04:00:00,5200663,NaN,NaN,NaN,NaN,80 Sheppard Ave E,North York,M2N 6E8,"{u'type': u'Point', u'coordinates': (-79.40809..."
4,5,266,Academy of Computer and Employment Skills,"55 Eglinton Ave E, Ste 703, Toronto, ON M4...","Mount Pleasant West, 104",416-703-7770 ext 206,marina.n@workingskillscentre.com,"<a href=""http://www.workingskillscentre.com/di...",Canadian Language Benchmark Level 4 to Level 7...,Registered private career college * vocational...,...,2013-12-06T05:00:00,800592,NaN,NaN,NaN,NaN,55 Eglinton Ave E,former Toronto,M4P 1G8,"{u'type': u'Point', u'coordinates': (-79.39607..."


In [3]:
#list column names
list(df.columns) 

['_id',
 'OBJECTID',
 'AGENCY_NAME',
 'ORGANIZATION_ADDRESS',
 'NEIGHBOURHOOD',
 'OFFICE_PHONE',
 'EMAIL',
 'WEBSITE',
 'ELIGIBILITY',
 'DESCRIPTION_SERVICE',
 'APPLICATION',
 'LANGUAGES',
 'ACCESSIBILITY',
 'HOURS',
 'LEGAL_STATUS',
 'DATE_UPDATED',
 'ADDRESS_POINT_ID',
 'X',
 'Y',
 'LONGITUDE',
 'LATITUDE',
 'ADDRESS_FULL',
 'MUNICIPALITY',
 'POSTAL_CODE',
 'geometry']

In [4]:
#Simplify the data by deleting columns that we do not need
columns = ['_id', 'OBJECTID','OFFICE_PHONE','EMAIL', 'WEBSITE','ELIGIBILITY','APPLICATION','LANGUAGES','ACCESSIBILITY','HOURS','LEGAL_STATUS','DATE_UPDATED','ADDRESS_POINT_ID','X','Y']
df.drop(columns, inplace=True, axis=1)
df.head()

,AGENCY_NAME,ORGANIZATION_ADDRESS,NEIGHBOURHOOD,DESCRIPTION_SERVICE,LONGITUDE,LATITUDE,ADDRESS_FULL,MUNICIPALITY,POSTAL_CODE,geometry
0,Joint Training and Apprenticeship Committee,"936 Warden Ave, Toronto, ON M1L 4C9","Wexford-Maryvale, 119",Apprenticeship programs in plumbing and steamf...,NaN,NaN,936 Warden Ave,Scarborough,M1L 4C9,"{u'type': u'Point', u'coordinates': (-79.29001..."
1,Ontario Industrial and Finishing Skills Centre,"130 Toro Rd, Unit C, Toronto, ON M3J 3M9","York University Heights, 27",Apprenticeship programs for painter decorators...,NaN,NaN,130 Toro Rd,North York,M3J 3M9,"{u'type': u'Point', u'coordinates': (-79.48321..."
2,Toronto Ironworkers Local 721,"909 Kipling Ave, Toronto, ON M8Z 5H3","Islington-City Centre West, 14",Apprenticeship programs for ironworkers and ro...,NaN,NaN,909 Kipling Ave,Etobicoke,M8Z 5H3,"{u'type': u'Point', u'coordinates': (-79.53300..."
3,Toronto Catholic District School Board,"Catholic Education Centre, 80 Sheppard Ave E, ...","Willowdale East, 51",Full academic program in a Roman Catholic envi...,NaN,NaN,80 Sheppard Ave E,North York,M2N 6E8,"{u'type': u'Point', u'coordinates': (-79.40809..."
4,Academy of Computer and Employment Skills,"55 Eglinton Ave E, Ste 703, Toronto, ON M4...","Mount Pleasant West, 104",Registered private career college * vocational...,NaN,NaN,55 Eglinton Ave E,former Toronto,M4P 1G8,"{u'type': u'Point', u'coordinates': (-79.39607..."


Next, we need to fill the data in 'Longitude' & 'Latitude' column based on the information in the 'geometry' column, and then delete the 'geometry' column to simplify the table.

In [5]:
#get a value from a cell of a 'geometry' column to see its format
value = df.iloc[0]["geometry"]
print(value)

{u'type': u'Point', u'coordinates': (-79.2900147896, 43.7315309099)}


In [6]:
#remove unnecessary character in 'geometry' data column
spec_chars = ["{",'u',"'","type",":","Point",",","coordinates","(",")","}"]
for char in spec_chars:
    df['geometry'] = df['geometry'].str.replace(char, ' ')

In [7]:
df.head() #check the new df format

,AGENCY_NAME,ORGANIZATION_ADDRESS,NEIGHBOURHOOD,DESCRIPTION_SERVICE,LONGITUDE,LATITUDE,ADDRESS_FULL,MUNICIPALITY,POSTAL_CODE,geometry
0,Joint Training and Apprenticeship Committee,"936 Warden Ave, Toronto, ON M1L 4C9","Wexford-Maryvale, 119",Apprenticeship programs in plumbing and steamf...,NaN,NaN,936 Warden Ave,Scarborough,M1L 4C9,-79.2900147896 43.7315309...
1,Ontario Industrial and Finishing Skills Centre,"130 Toro Rd, Unit C, Toronto, ON M3J 3M9","York University Heights, 27",Apprenticeship programs for painter decorators...,NaN,NaN,130 Toro Rd,North York,M3J 3M9,-79.4832149226 43.7619063...
2,Toronto Ironworkers Local 721,"909 Kipling Ave, Toronto, ON M8Z 5H3","Islington-City Centre West, 14",Apprenticeship programs for ironworkers and ro...,NaN,NaN,909 Kipling Ave,Etobicoke,M8Z 5H3,-79.5330070292 43.6376791...
3,Toronto Catholic District School Board,"Catholic Education Centre, 80 Sheppard Ave E, ...","Willowdale East, 51",Full academic program in a Roman Catholic envi...,NaN,NaN,80 Sheppard Ave E,North York,M2N 6E8,-79.408095975 43.76284409...
4,Academy of Computer and Employment Skills,"55 Eglinton Ave E, Ste 703, Toronto, ON M4...","Mount Pleasant West, 104",Registered private career college * vocational...,NaN,NaN,55 Eglinton Ave E,former Toronto,M4P 1G8,-79.3960778236 43.7069069...


In [8]:
#get a value from a cell of a 'geometry' column again to check its format
value = df.iloc[0]["geometry"]
print(value)

                    -79.2900147896  43.7315309099  


In [9]:
#Fill the data for 'Longitude' and 'Latitude' column based on values in 'geometry' column
#build'new' data frame with split value columns 
new = df["geometry"].str.split(" ", n = 1, expand = True) 

# making separate longitude column from new data frame 
df["LONGITUDE"]= new[0] 

# making separate latitude column from new data frame 
df["LATITUDE"]= new[1]

# Dropping old Name columns 
df.drop(columns =["geometry"], inplace = True) 
  
# df display 
df

,AGENCY_NAME,ORGANIZATION_ADDRESS,NEIGHBOURHOOD,DESCRIPTION_SERVICE,LONGITUDE,LATITUDE,ADDRESS_FULL,MUNICIPALITY,POSTAL_CODE
0,Joint Training and Apprenticeship Committee,"936 Warden Ave, Toronto, ON M1L 4C9","Wexford-Maryvale, 119",Apprenticeship programs in plumbing and steamf...,,-79.2900147896 43.73153090...,936 Warden Ave,Scarborough,M1L 4C9
1,Ontario Industrial and Finishing Skills Centre,"130 Toro Rd, Unit C, Toronto, ON M3J 3M9","York University Heights, 27",Apprenticeship programs for painter decorators...,,-79.4832149226 43.76190630...,130 Toro Rd,North York,M3J 3M9
2,Toronto Ironworkers Local 721,"909 Kipling Ave, Toronto, ON M8Z 5H3","Islington-City Centre West, 14",Apprenticeship programs for ironworkers and ro...,,-79.5330070292 43.63767918...,909 Kipling Ave,Etobicoke,M8Z 5H3
3,Toronto Catholic District School Board,"Catholic Education Centre, 80 Sheppard Ave E, ...","Willowdale East, 51",Full academic program in a Roman Catholic envi...,,-79.408095975 43.7628440941,80 Sheppard Ave E,North York,M2N 6E8
4,Academy of Computer and Employment Skills,"55 Eglinton Ave E, Ste 703, Toronto, ON M4...","Mount Pleasant West, 104",Registered private career college * vocational...,,-79.3960778236 43.70690691...,55 Eglinton Ave E,former Toronto,M4P 1G8
5,Redemption Reintegration Services,"1460 Midland Ave, Toronto, ON M1P 3B9","Dorset Park, 126",Reintegration services for African/Caribbean y...,,-79.2656257941 43.75238817...,1460 Midland Ave,Scarborough,M1P 3B9
6,Conseil scolaire Viamonde,"116 Cornelius Pkwy, Toronto, ON M6L 2K5","Maple Leaf, 29","Academic, commercial and technical day courses...",,-79.4755347034 43.71782792...,116 Cornelius Pkwy,North York,M6L 2K5
7,Toronto District School Board,"Head Office, 5050 Yonge St, Toronto, ON M2...","Lansing-Westgate, 38","Academic, commercial and technical, day and ni...",,-79.4134194857 43.76704859...,5050 Yonge St,North York,M2N 5N8
8,Humber College,"Humber North Campus, 205 Humber College Blvd, ...","West Humber-Clairville, 1","Full and part time diploma, certificate, appre...",,-79.6068398627 43.72916654...,205 Humber College Blvd,Etobicoke,M9W 5L7
9,Seneca College,"Newnham Campus, 1750 Finch Ave E, Toronto, ON ...","Don Valley Village, 47",Full and part time diploma and certificate pro...,,-79.3489876443 43.79578080...,1750 Finch Ave E,North York,M2J 2X5


In [10]:
#Oops, the code above does not work as I expected haha...
#Let's use the str.split function to take the longitude value from the latitude column
df[['LONGITUDE','LATITUDE']] = df.LATITUDE.str.split(expand=True) 

In [11]:
# df display 
df

,AGENCY_NAME,ORGANIZATION_ADDRESS,NEIGHBOURHOOD,DESCRIPTION_SERVICE,LONGITUDE,LATITUDE,ADDRESS_FULL,MUNICIPALITY,POSTAL_CODE
0,Joint Training and Apprenticeship Committee,"936 Warden Ave, Toronto, ON M1L 4C9","Wexford-Maryvale, 119",Apprenticeship programs in plumbing and steamf...,-79.2900147896,43.7315309099,936 Warden Ave,Scarborough,M1L 4C9
1,Ontario Industrial and Finishing Skills Centre,"130 Toro Rd, Unit C, Toronto, ON M3J 3M9","York University Heights, 27",Apprenticeship programs for painter decorators...,-79.4832149226,43.7619063057,130 Toro Rd,North York,M3J 3M9
2,Toronto Ironworkers Local 721,"909 Kipling Ave, Toronto, ON M8Z 5H3","Islington-City Centre West, 14",Apprenticeship programs for ironworkers and ro...,-79.5330070292,43.6376791873,909 Kipling Ave,Etobicoke,M8Z 5H3
3,Toronto Catholic District School Board,"Catholic Education Centre, 80 Sheppard Ave E, ...","Willowdale East, 51",Full academic program in a Roman Catholic envi...,-79.408095975,43.7628440941,80 Sheppard Ave E,North York,M2N 6E8
4,Academy of Computer and Employment Skills,"55 Eglinton Ave E, Ste 703, Toronto, ON M4...","Mount Pleasant West, 104",Registered private career college * vocational...,-79.3960778236,43.7069069156,55 Eglinton Ave E,former Toronto,M4P 1G8
5,Redemption Reintegration Services,"1460 Midland Ave, Toronto, ON M1P 3B9","Dorset Park, 126",Reintegration services for African/Caribbean y...,-79.2656257941,43.7523881717,1460 Midland Ave,Scarborough,M1P 3B9
6,Conseil scolaire Viamonde,"116 Cornelius Pkwy, Toronto, ON M6L 2K5","Maple Leaf, 29","Academic, commercial and technical day courses...",-79.4755347034,43.7178279267,116 Cornelius Pkwy,North York,M6L 2K5
7,Toronto District School Board,"Head Office, 5050 Yonge St, Toronto, ON M2...","Lansing-Westgate, 38","Academic, commercial and technical, day and ni...",-79.4134194857,43.7670485971,5050 Yonge St,North York,M2N 5N8
8,Humber College,"Humber North Campus, 205 Humber College Blvd, ...","West Humber-Clairville, 1","Full and part time diploma, certificate, appre...",-79.6068398627,43.7291665492,205 Humber College Blvd,Etobicoke,M9W 5L7
9,Seneca College,"Newnham Campus, 1750 Finch Ave E, Toronto, ON ...","Don Valley Village, 47",Full and part time diploma and certificate pro...,-79.3489876443,43.7957808076,1750 Finch Ave E,North York,M2J 2X5


In [12]:
#Count how many universities/ learning centres are in each street/ neighbourhood
df.groupby('NEIGHBOURHOOD')['AGENCY_NAME'].count().sort_values(ascending=False)

NEIGHBOURHOOD
York University Heights, 27       2
Willowdale East, 51               1
Wexford-Maryvale, 119             1
West Humber-Clairville, 1         1
University, 79                    1
North Riverdale, 68               1
Newtonbrook West, 36              1
Mount Pleasant West, 104          1
Maple Leaf, 29                    1
Lansing-Westgate, 38              1
Islington-City Centre West, 14    1
Downsview-Roding-CFB, 26          1
Dorset Park, 126                  1
Don Valley Village, 47            1
Church-Yonge Corridor, 75         1
Name: AGENCY_NAME, dtype: int64

In [13]:
#Count how many universities/ learning centres are in each borough
df.groupby('MUNICIPALITY')['AGENCY_NAME'].count().sort_values(ascending=False)

MUNICIPALITY
North York        8
former Toronto    4
Scarborough       2
Etobicoke         2
Name: AGENCY_NAME, dtype: int64

We find out that
* There are **16 universities/ learning centres in Toronto City spread in 15 Neighbourhoods**, where there are 2 universities/ learning centres in the neighbourhood area of **York University, Heights, 27**, while in other neighbourhoods each as 1 university/ learning centre
* Borough that has the most university/ learning centre are **North York (8 universities/ learning centres)** and **Toronto (4 universities/learning centers)**, these two boroughs are good candidates for our coffee shop location as we target students as our main customers

### 4b. Exploring The Neighbourhood

It's time for us to explore the targetted area using a map to find out about the distance of the universities/ learning centres and how many coffee shops are currently exist around the universities/ learning centres

In [14]:
#download all the dependencies that we will need to explore the neighbourhood

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium==0.5
import folium

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

#code below to install folium gives error
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [15]:
df.dtypes #check data type

AGENCY_NAME             object
ORGANIZATION_ADDRESS    object
NEIGHBOURHOOD           object
DESCRIPTION_SERVICE     object
LONGITUDE               object
LATITUDE                object
ADDRESS_FULL            object
MUNICIPALITY            object
POSTAL_CODE             object
dtype: object

In [16]:
#change data type longitude and latitute into float
df[["LONGITUDE","LATITUDE"]] = df[["LONGITUDE", "LATITUDE"]].astype("float")

In [17]:
# create Toronto, CA map using the latitude and longitude values
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [18]:
# create map using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood, school in zip(df['LATITUDE'], df['LONGITUDE'], df['MUNICIPALITY'], df['NEIGHBOURHOOD'], df['AGENCY_NAME']):
    label = '{}, {}, {}'.format(neighborhood, borough, school)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

After putting the longitude & latitude  in the map, we can get a picture regarding the distance among universities/ learning centers in Toronto City, and easily notice that there are clusters. We also can see that :
* In **North York borough**, there are neighbourhoods that have their universities/ learning centre close to each other :
 <br>
  <br>
  a. Lansing - Westgate, 38 (Toronto District School Board) & Willowdale East, 51 (Toronto Catholic District School Board)<br>
  b. Downsview-Roading-CFB, 26 (LIUNA Local Training Centre) & Maple Leaf, 29 (Conseil Scolaire Viamonde) <br>
  c. York University Heights, 27 (York University) & York University Heights, 27 (Ontario Industrial and Finishing Skills Centre)
   <br><br>
* In **Former Toronto borough**, there is also neighbourhood that has its universities/ learning centre close to each other :
 <br>
  <br>
  a. Church-Yonge Corridor, 75 (Ryerson University) & University, 79 (University of Toronto)
  
Next, let's focus on these learning centres and these two boroughs. Since they have more universities/ learning centres, we assume that these areas are visited often by students. We need to find out how many coffee shops are currently exist in the area to measure the competition level and find which areas/ markets are currently underserved.

### 4C. Explore The Coffee Shops Near The Targetted Universities/ Learning Centres

In [19]:
# The code was removed by Watson Studio for sharing.

In [20]:
#Set 'AGENCY_NAME' column as index on the dataframe 
df.set_index("AGENCY_NAME", inplace = True) 

### 4C.1 Explore Learning Centres Area In North York (1st Street Cluster)

1. Toronto District School Board in Lansing - Westgate, 38

In [21]:
#Using the operator .loc[] to select data and see the longitude & latitude of Toronto District School Board
toronto_lansing = df.loc["Toronto District School Board"]
toronto_lansing

ORGANIZATION_ADDRESS    Head Office, 5050 Yonge St, Toronto, ON     M2...
NEIGHBOURHOOD                                        Lansing-Westgate, 38
DESCRIPTION_SERVICE     Academic, commercial and technical, day and ni...
LONGITUDE                                                        -79.4134
LATITUDE                                                           43.767
ADDRESS_FULL                                                5050 Yonge St
MUNICIPALITY                                                   North York
POSTAL_CODE                                                       M2N 5N8
Name: Toronto District School Board, dtype: object

In [22]:
#Get the long lat data from the table
toronto_lansing_latitude = toronto_lansing['LATITUDE']
toronto_lansing_longitude = toronto_lansing['LONGITUDE']
print('Latitude and longitude values of Toronto District School Board are {}, {}.'.format(toronto_lansing_latitude, 
                                                               toronto_lansing_longitude))

Latitude and longitude values of Toronto District School Board are 43.7670485971, -79.4134194857.


In [23]:
#Explore coffee shops around Toronto District School Board with radius 1Km
radius = 1000
search_query = 'Coffee'
# create URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    toronto_lansing_latitude, 
    toronto_lansing_longitude,
    VERSION,
    search_query,
    radius,
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/search?client_id=3KZIEIWOYFJZIM3SOLIV3EABZP5XOTOVVQVBJJF2CPH4ECOE&client_secret=PJZ5VDM0A2I1JDQDDVXJ2XIGHXPLH4VVIPC0EREVOCNQSE4D&ll=43.7670485971,-79.4134194857&v=20180604&query=Coffee&radius=1000&limit=30'

In [24]:
#Send the GET Request and examine the results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fd1cd211d98ae664e7e997f'},
 'response': {'venues': [{'id': '58a5d1e5739d851f1899abc9',
    'name': 'WFM Coffee Bar',
    'location': {'address': '4771 Yonge Street',
     'lat': 43.76046375,
     'lng': -79.41066125,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.76046375,
       'lng': -79.41066125}],
     'distance': 765,
     'postalCode': 'M2N 5M5',
     'cc': 'CA',
     'city': 'North York',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['4771 Yonge Street',
      'North York ON M2N 5M5',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d1e0931735',
      'name': 'Coffee Shop',
      'pluralName': 'Coffee Shops',
      'shortName': 'Coffee Shop',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1607585057',
    'hasPerk': False},
   {'id': '50b6489be4b0bcc63e8fff31',
    'name': 'Little It

In [25]:
#Get the relevant part of JSON and transform it into a pandas dataframe
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet
0,58a5d1e5739d851f1899abc9,WFM Coffee Bar,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1607585057,False,4771 Yonge Street,43.760464,-79.410661,"[{'label': 'display', 'lat': 43.76046375, 'lng...",765,M2N 5M5,CA,North York,ON,Canada,"[4771 Yonge Street, North York ON M2N 5M5, Can...",NaN
1,50b6489be4b0bcc63e8fff31,Little Italy Coffee Shop,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1607585057,False,NaN,43.767728,-79.412739,"[{'label': 'display', 'lat': 43.76772773537328...",93,NaN,CA,NaN,NaN,Canada,[Canada],NaN
2,5c86b6fd356b49002cf7d580,Second Cup Coffee Co. featuring Pinkberry Froz...,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1607585057,False,5150 Yonge Street,43.768693,-79.413802,"[{'label': 'display', 'lat': 43.76869256676924...",185,M2N 6L8,CA,Toronto,ON,Canada,"[5150 Yonge Street, Toronto ON M2N 6L8, Canada]",NaN
3,4c1da44663750f4796afb867,Coffee Plus,[],v-1607585057,False,NaN,43.775165,-79.412964,"[{'label': 'display', 'lat': 43.775165, 'lng':...",904,NaN,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",Yonge & Finch
4,4c13ab787f7f2d7fc06adf68,Coffee Time,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1607585057,False,NaN,43.763900,-79.399310,"[{'label': 'display', 'lat': 43.7639002546235,...",1187,NaN,CA,NaN,NaN,Canada,[Canada],NaN


In [26]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,WFM Coffee Bar,Coffee Shop,4771 Yonge Street,43.760464,-79.410661,"[{'label': 'display', 'lat': 43.76046375, 'lng...",765,M2N 5M5,CA,North York,ON,Canada,"[4771 Yonge Street, North York ON M2N 5M5, Can...",NaN,58a5d1e5739d851f1899abc9
1,Little Italy Coffee Shop,Coffee Shop,NaN,43.767728,-79.412739,"[{'label': 'display', 'lat': 43.76772773537328...",93,NaN,CA,NaN,NaN,Canada,[Canada],NaN,50b6489be4b0bcc63e8fff31
2,Second Cup Coffee Co. featuring Pinkberry Froz...,Café,5150 Yonge Street,43.768693,-79.413802,"[{'label': 'display', 'lat': 43.76869256676924...",185,M2N 6L8,CA,Toronto,ON,Canada,"[5150 Yonge Street, Toronto ON M2N 6L8, Canada]",NaN,5c86b6fd356b49002cf7d580
3,Coffee Plus,None,NaN,43.775165,-79.412964,"[{'label': 'display', 'lat': 43.775165, 'lng':...",904,NaN,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",Yonge & Finch,4c1da44663750f4796afb867
4,Coffee Time,Coffee Shop,NaN,43.763900,-79.399310,"[{'label': 'display', 'lat': 43.7639002546235,...",1187,NaN,CA,NaN,NaN,Canada,[Canada],NaN,4c13ab787f7f2d7fc06adf68
5,Coffee Jazz Mixers,Coffee Shop,28 Finch Ave West,43.779243,-79.418434,"[{'label': 'display', 'lat': 43.77924346923828...",1416,M2N 2G7,CA,North York,ON,Canada,"[28 Finch Ave West, North York ON M2N 2G7, Can...",NaN,534ab00f498ed9825404cd56
6,Another Land Coffee And More,Coffee Shop,4714 Yonge Street,43.759309,-79.410855,"[{'label': 'display', 'lat': 43.759309, 'lng':...",885,M2N 5M4,CA,Toronto,ON,Canada,"[4714 Yonge Street (Sheppard Ave), Toronto ON ...",Sheppard Ave,5d5b7a9d912a95000758f395
7,U9 Coffee And Tea,Coffee Shop,5 Everson Drive,43.757229,-79.405945,"[{'label': 'display', 'lat': 43.75722869590154...",1247,NaN,CA,Toronto,ON,Canada,"[5 Everson Drive, Toronto ON, Canada]",NaN,4f06526fb63440fe1fec41aa
8,Starbucks,Coffee Shop,180 Queen Street West,43.766994,-79.410788,"[{'label': 'display', 'lat': 43.76699378673400...",211,NaN,CA,Toronto,ON,Canada,"[180 Queen Street West (University Ave.), Toro...",University Ave.,50b137d1e4b03c6b1e792132
9,Starbucks,Coffee Shop,5140 Yonge St,43.768353,-79.413046,"[{'label': 'display', 'lat': 43.768353, 'lng':...",148,M2N 6L7,CA,Toronto,ON,Canada,"[5140 Yonge St (at Park Home Ave), Toronto ON ...",at Park Home Ave,4aedfeadf964a52005d121e3


In [27]:
dataframe_filtered.name

0                                       WFM Coffee Bar
1                             Little Italy Coffee Shop
2    Second Cup Coffee Co. featuring Pinkberry Froz...
3                                          Coffee Plus
4                                          Coffee Time
5                                   Coffee Jazz Mixers
6                         Another Land Coffee And More
7                                    U9 Coffee And Tea
8                                            Starbucks
9                                            Starbucks
Name: name, dtype: object

In [28]:
#display the nearest coffee shops in the map
venues_map = folium.Map(location=[toronto_lansing_latitude,toronto_lansing_longitude,], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Toronto District School Board
folium.CircleMarker(
    [toronto_lansing_latitude, toronto_lansing_longitude],
    radius=10,
    color='red',
    popup='Toronto District School Board',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the coffee shops as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

* **There are 4 coffee shops < 500** m from Toronto District School Board : Little Italy Coffee Shop, Second Cup Coffee Co., and 2 Starbucks 
* **There are 6 coffee shops in the radius of < 1.5 Km** : WFM Coffee Bar, Coffee Plus, Coffee Time, Coffee Jazz Mixers, Another Land Coffee And More, U9 Coffee And Tea

2. Toronto Catholic District School Board in Willowdale East, 51

In [29]:
#Using the operator .loc[] to select data and see the longitude & latitude of Toronto District School Board
toronto_willow = df.loc["Toronto Catholic District School Board"]
toronto_willow

ORGANIZATION_ADDRESS    Catholic Education Centre, 80 Sheppard Ave E, ...
NEIGHBOURHOOD                                         Willowdale East, 51
DESCRIPTION_SERVICE     Full academic program in a Roman Catholic envi...
LONGITUDE                                                        -79.4081
LATITUDE                                                          43.7628
ADDRESS_FULL                                            80 Sheppard Ave E
MUNICIPALITY                                                   North York
POSTAL_CODE                                                       M2N 6E8
Name: Toronto Catholic District School Board, dtype: object

In [30]:
toronto_willow_latitude = toronto_willow['LATITUDE']
toronto_willow_longitude = toronto_willow['LONGITUDE']
print('Latitude and longitude values of Toronto Catholic District School Board are {}, {}.'.format(toronto_willow_latitude, 
                                                               toronto_willow_longitude))

Latitude and longitude values of Toronto Catholic District School Board are 43.7628440941, -79.408095975.


In [31]:
#Explore coffee shops around Toronto Catholic District School Board with radius 1Km
radius = 1000
search_query = 'Coffee'
# create URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    toronto_willow_latitude, 
    toronto_willow_longitude,
    VERSION,
    search_query,
    radius,
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/search?client_id=3KZIEIWOYFJZIM3SOLIV3EABZP5XOTOVVQVBJJF2CPH4ECOE&client_secret=PJZ5VDM0A2I1JDQDDVXJ2XIGHXPLH4VVIPC0EREVOCNQSE4D&ll=43.7628440941,-79.408095975&v=20180604&query=Coffee&radius=1000&limit=30'

In [32]:
#Send the GET Request and examine the results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fd1cd218cc76a4d450297e8'},
 'response': {'venues': [{'id': '58a5d1e5739d851f1899abc9',
    'name': 'WFM Coffee Bar',
    'location': {'address': '4771 Yonge Street',
     'lat': 43.76046375,
     'lng': -79.41066125,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.76046375,
       'lng': -79.41066125}],
     'distance': 335,
     'postalCode': 'M2N 5M5',
     'cc': 'CA',
     'city': 'North York',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['4771 Yonge Street',
      'North York ON M2N 5M5',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d1e0931735',
      'name': 'Coffee Shop',
      'pluralName': 'Coffee Shops',
      'shortName': 'Coffee Shop',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1607585057',
    'hasPerk': False},
   {'id': '4c13ab787f7f2d7fc06adf68',
    'name': 'Coffee Ti

In [33]:
#Get the relevant part of JSON and transform it into a pandas dataframe assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet
0,58a5d1e5739d851f1899abc9,WFM Coffee Bar,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1607585057,False,4771 Yonge Street,43.760464,-79.410661,"[{'label': 'display', 'lat': 43.76046375, 'lng...",335,M2N 5M5,CA,North York,ON,Canada,"[4771 Yonge Street, North York ON M2N 5M5, Can...",NaN
1,4c13ab787f7f2d7fc06adf68,Coffee Time,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1607585057,False,NaN,43.763900,-79.399310,"[{'label': 'display', 'lat': 43.7639002546235,...",716,NaN,CA,NaN,NaN,Canada,[Canada],NaN
2,5d5b7a9d912a95000758f395,Another Land Coffee And More,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1607585057,False,4714 Yonge Street,43.759309,-79.410855,"[{'label': 'display', 'lat': 43.759309, 'lng':...",451,M2N 5M4,CA,Toronto,ON,Canada,"[4714 Yonge Street (Sheppard Ave), Toronto ON ...",Sheppard Ave
3,4f06526fb63440fe1fec41aa,U9 Coffee And Tea,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1607585057,False,5 Everson Drive,43.757229,-79.405945,"[{'label': 'display', 'lat': 43.75722869590154...",648,NaN,CA,Toronto,ON,Canada,"[5 Everson Drive, Toronto ON, Canada]",NaN
4,50b6489be4b0bcc63e8fff31,Little Italy Coffee Shop,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1607585057,False,NaN,43.767728,-79.412739,"[{'label': 'display', 'lat': 43.76772773537328...",659,NaN,CA,NaN,NaN,Canada,[Canada],NaN


In [34]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,WFM Coffee Bar,Coffee Shop,4771 Yonge Street,43.760464,-79.410661,"[{'label': 'display', 'lat': 43.76046375, 'lng...",335,M2N 5M5,CA,North York,ON,Canada,"[4771 Yonge Street, North York ON M2N 5M5, Can...",NaN,58a5d1e5739d851f1899abc9
1,Coffee Time,Coffee Shop,NaN,43.763900,-79.399310,"[{'label': 'display', 'lat': 43.7639002546235,...",716,NaN,CA,NaN,NaN,Canada,[Canada],NaN,4c13ab787f7f2d7fc06adf68
2,Another Land Coffee And More,Coffee Shop,4714 Yonge Street,43.759309,-79.410855,"[{'label': 'display', 'lat': 43.759309, 'lng':...",451,M2N 5M4,CA,Toronto,ON,Canada,"[4714 Yonge Street (Sheppard Ave), Toronto ON ...",Sheppard Ave,5d5b7a9d912a95000758f395
3,U9 Coffee And Tea,Coffee Shop,5 Everson Drive,43.757229,-79.405945,"[{'label': 'display', 'lat': 43.75722869590154...",648,NaN,CA,Toronto,ON,Canada,"[5 Everson Drive, Toronto ON, Canada]",NaN,4f06526fb63440fe1fec41aa
4,Little Italy Coffee Shop,Coffee Shop,NaN,43.767728,-79.412739,"[{'label': 'display', 'lat': 43.76772773537328...",659,NaN,CA,NaN,NaN,Canada,[Canada],NaN,50b6489be4b0bcc63e8fff31
5,Coffee Plus,None,NaN,43.775165,-79.412964,"[{'label': 'display', 'lat': 43.775165, 'lng':...",1426,NaN,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",Yonge & Finch,4c1da44663750f4796afb867
6,Second Cup Coffee Co. featuring Pinkberry Froz...,Café,5150 Yonge Street,43.768693,-79.413802,"[{'label': 'display', 'lat': 43.76869256676924...",796,M2N 6L8,CA,Toronto,ON,Canada,"[5150 Yonge Street, Toronto ON M2N 6L8, Canada]",NaN,5c86b6fd356b49002cf7d580
7,Starbucks,Coffee Shop,180 Queen Street West,43.766994,-79.410788,"[{'label': 'display', 'lat': 43.76699378673400...",510,NaN,CA,Toronto,ON,Canada,"[180 Queen Street West (University Ave.), Toro...",University Ave.,50b137d1e4b03c6b1e792132
8,Starbucks,Coffee Shop,5140 Yonge St,43.768353,-79.413046,"[{'label': 'display', 'lat': 43.768353, 'lng':...",731,M2N 6L7,CA,Toronto,ON,Canada,"[5140 Yonge St (at Park Home Ave), Toronto ON ...",at Park Home Ave,4aedfeadf964a52005d121e3


In [35]:
venues_map = folium.Map(location=[toronto_willow_latitude,toronto_willow_longitude,], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Toronto District School Board
folium.CircleMarker(
    [toronto_willow_latitude, toronto_willow_longitude],
    radius=10,
    color='red',
    popup='Toronto Catholic District School Board',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the coffee shops as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

* It looks like **the coffee shops near Toronto Catholic District School Board also serves Toronto Districk School, and all of the coffee shops are quite reachable from the location since all of them are on the radius < 1 Km**
* There are 3 coffee shops with radius < 500 m : WFM Coffee Bar	Coffee Shop, Another Land Coffee And More, and 1 Starbucks
* There are 5 coffee shops within the range of 600 m - 1 Km : Coffee Time,U9 Coffee And Tea,Little Italy Coffee Shop, Second Cup Coffee, and another Starbucks
* **Unlike Toronto District School Board, Toronto Catholic District School Board does not have any coffee shops in the radius of 100 m**

***
<font size="3"><font color='blue'> There are 10 coffee shops that will be our competitors if we open a new coffee shop in this street area. It looks like these streets are already well-served with coffee shops. Let's check other clusters.</font></font>
***

### 4C.2 Explore Learning Centres Area In North York (2nd Street Cluster)

1. LIUNA Local Training Centre in Downsview-Roading-CFB, 26 

In [36]:
#Using the operator .loc[] to select data and see the longitude & latitude 
liuna_downsview = df.loc["LIUNA Local 183 Training Centre"]
liuna_downsview

ORGANIZATION_ADDRESS    1263 Wilson Ave, Ste 301, Toronto, ON     M3M 3G2
NEIGHBOURHOOD                                    Downsview-Roding-CFB, 26
DESCRIPTION_SERVICE     Apprenticeship programs in brick and stone mas...
LONGITUDE                                                        -79.4916
LATITUDE                                                          43.7243
ADDRESS_FULL                                              1263 Wilson Ave
MUNICIPALITY                                                   North York
POSTAL_CODE                                                       M3M 3G2
Name: LIUNA Local 183 Training Centre, dtype: object

In [37]:
liuna_downsview_latitude = liuna_downsview['LATITUDE']
liuna_downsview_longitude = liuna_downsview['LONGITUDE']
print('Latitude and longitude values of LIUNA Local Training Centre in Downsview-Roading-CFB, 26 are {}, {}.'.format(liuna_downsview_latitude, 
                                                               liuna_downsview_longitude))

Latitude and longitude values of LIUNA Local Training Centre in Downsview-Roading-CFB, 26 are 43.7242755262, -79.4915887618.


In [38]:
#Explore coffee shops around LIUNA Local 183 Training Centre with radius 1Km
radius = 1000
search_query = 'Coffee'
# create URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    liuna_downsview_latitude, 
    liuna_downsview_longitude,
    VERSION,
    search_query,
    radius,
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/search?client_id=3KZIEIWOYFJZIM3SOLIV3EABZP5XOTOVVQVBJJF2CPH4ECOE&client_secret=PJZ5VDM0A2I1JDQDDVXJ2XIGHXPLH4VVIPC0EREVOCNQSE4D&ll=43.7242755262,-79.4915887618&v=20180604&query=Coffee&radius=1000&limit=30'

In [39]:
#Send the GET Request and examine the results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fd1cd2155f62738f6a0bdce'},
 'response': {'venues': [{'id': '4d46aec2c3e5f04db91fb020',
    'name': 'Coffee Time',
    'location': {'lat': 43.72851501,
     'lng': -79.47792573,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.72851501,
       'lng': -79.47792573}],
     'distance': 1196,
     'cc': 'CA',
     'country': 'Canada',
     'formattedAddress': ['Canada']},
    'categories': [{'id': '4bf58dd8d48988d16d941735',
      'name': 'Café',
      'pluralName': 'Cafés',
      'shortName': 'Café',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1607585057',
    'hasPerk': False}]}}

In [40]:
#Get the relevant part of JSON and transform it into a pandas dataframe assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.country,location.formattedAddress
0,4d46aec2c3e5f04db91fb020,Coffee Time,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1607585057,False,43.728515,-79.477926,"[{'label': 'display', 'lat': 43.72851501, 'lng...",1196,CA,Canada,[Canada]


In [41]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,lat,lng,labeledLatLngs,distance,cc,country,formattedAddress,id
0,Coffee Time,Café,43.728515,-79.477926,"[{'label': 'display', 'lat': 43.72851501, 'lng...",1196,CA,Canada,[Canada],4d46aec2c3e5f04db91fb020


In [42]:
venues_map = folium.Map(location=[liuna_downsview_latitude,liuna_downsview_longitude,], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Toronto District School Board
folium.CircleMarker(
    [liuna_downsview_latitude, liuna_downsview_longitude],
    radius=10,
    color='red',
    popup='LIUNA Local Training Centre',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the coffee shops as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

* **There is only 1 coffee shop near LIUNA Local Training Centre** : Coffee Time with the distance 1.2 Km. **This is a potential area where we can build our coffee shop since there are no coffee shops close to the learning centre.**

 2. Conseil Scolaire Viamonde in Maple Leaf, 29

In [43]:
#Using the operator .loc[] to select data and see the longitude & latitude 
conseil_maple = df.loc["Conseil scolaire Viamonde"]
conseil_maple

ORGANIZATION_ADDRESS          116 Cornelius Pkwy, Toronto, ON     M6L 2K5
NEIGHBOURHOOD                                              Maple Leaf, 29
DESCRIPTION_SERVICE     Academic, commercial and technical day courses...
LONGITUDE                                                        -79.4755
LATITUDE                                                          43.7178
ADDRESS_FULL                                           116 Cornelius Pkwy
MUNICIPALITY                                                   North York
POSTAL_CODE                                                       M6L 2K5
Name: Conseil scolaire Viamonde, dtype: object

In [44]:
conseil_maple_latitude = conseil_maple['LATITUDE']
conseil_maple_longitude = conseil_maple['LONGITUDE']
print('Latitude and longitude values of Conseil Scolaire Viamondeare {}, {}.'.format(conseil_maple_latitude, 
                                                               conseil_maple_longitude))

Latitude and longitude values of Conseil Scolaire Viamondeare 43.7178279267, -79.4755347034.


In [45]:
#Explore coffee shops around Conseil Scolaire Viamonde in Maple Leaf, 29 with radius 1Km
radius = 1000
search_query = 'Coffee'
# create URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    conseil_maple_latitude, 
    conseil_maple_longitude,
    VERSION,
    search_query,
    radius,
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/search?client_id=3KZIEIWOYFJZIM3SOLIV3EABZP5XOTOVVQVBJJF2CPH4ECOE&client_secret=PJZ5VDM0A2I1JDQDDVXJ2XIGHXPLH4VVIPC0EREVOCNQSE4D&ll=43.7178279267,-79.4755347034&v=20180604&query=Coffee&radius=1000&limit=30'

In [46]:
#Send the GET Request and examine the results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fd1cd22ea3ff877cf9ee80f'},
 'response': {'venues': [{'id': '4d46aec2c3e5f04db91fb020',
    'name': 'Coffee Time',
    'location': {'lat': 43.72851501,
     'lng': -79.47792573,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.72851501,
       'lng': -79.47792573}],
     'distance': 1205,
     'cc': 'CA',
     'country': 'Canada',
     'formattedAddress': ['Canada']},
    'categories': [{'id': '4bf58dd8d48988d16d941735',
      'name': 'Café',
      'pluralName': 'Cafés',
      'shortName': 'Café',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1607585058',
    'hasPerk': False}]}}

In [47]:
#Get the relevant part of JSON and transform it into a pandas dataframe assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.country,location.formattedAddress
0,4d46aec2c3e5f04db91fb020,Coffee Time,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1607585058,False,43.728515,-79.477926,"[{'label': 'display', 'lat': 43.72851501, 'lng...",1205,CA,Canada,[Canada]


In [48]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,lat,lng,labeledLatLngs,distance,cc,country,formattedAddress,id
0,Coffee Time,Café,43.728515,-79.477926,"[{'label': 'display', 'lat': 43.72851501, 'lng...",1205,CA,Canada,[Canada],4d46aec2c3e5f04db91fb020


In [49]:
venues_map = folium.Map(location=[liuna_downsview_latitude,liuna_downsview_longitude,], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Toronto District School Board
folium.CircleMarker(
    [conseil_maple_latitude, conseil_maple_longitude],
    radius=10,
    color='red',
    popup='Conseil Scolaire Viamonde',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the coffee shops as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.name):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

* **There is only 1 coffee shop near Conseil Scolaire Viamonde** : Coffee Time also with the distance 1.2 Km.

***
<font size="3"><font color='blue'> This street area can be our potential area to build our coffee shop for these reasons :<br>
<br> a. The street area has two learning centres nearby : LIUNA Local Training Centres & Conseil Scolaire Viamonde
<br> b. The street area is still underserved with only 1 coffee shop nearby, the competition level is low
<br> c. The distance to the nearest coffee shop (Coffee Time) is around 1.2 Km, if we build our coffee shop closer/more reachable from the learning centres, we can grab the students as our potential market.</font></font>
***

### 4C.3 Explore Learning Centres Area In North York (3rd Street Cluster)

1. York University in York University Heights, 27

In [50]:
#Using the operator .loc[] to select data and see the longitude & latitude 
york_university = df.loc["York University"]
york_university

ORGANIZATION_ADDRESS               4700 Keele St, Toronto, ON     M3J 1P3
NEIGHBOURHOOD                                 York University Heights, 27
DESCRIPTION_SERVICE     Full and part time undergraduate and graduate ...
LONGITUDE                                                        -79.5023
LATITUDE                                                          43.7734
ADDRESS_FULL                                                4700 Keele St
MUNICIPALITY                                                   North York
POSTAL_CODE                                                       M3J 1P3
Name: York University, dtype: object

In [51]:
york_university_latitude = york_university['LATITUDE']
york_university_longitude = york_university['LONGITUDE']
print('Latitude and longitude values of York University  are {}, {}.'.format(york_university_latitude, 
                                                               york_university_longitude))

Latitude and longitude values of York University  are 43.77341009, -79.5023381684.


In [52]:
#Explore coffee shops around LIUNA Local 183 Training Centre with radius 1Km
radius = 1000
search_query = 'Coffee'
# create URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    york_university_latitude, 
    york_university_longitude,
    VERSION,
    search_query,
    radius,
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/search?client_id=3KZIEIWOYFJZIM3SOLIV3EABZP5XOTOVVQVBJJF2CPH4ECOE&client_secret=PJZ5VDM0A2I1JDQDDVXJ2XIGHXPLH4VVIPC0EREVOCNQSE4D&ll=43.77341009,-79.5023381684&v=20180604&query=Coffee&radius=1000&limit=30'

In [53]:
#Send the GET Request and examine the results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fd1cd224fbdfc4a2d057a21'},
 'response': {'venues': [{'id': '4c878e539c808cfa582e151e',
    'name': "Timothy's World Coffee",
    'location': {'address': '4700 Keele St.',
     'crossStreet': 'at Schulich School of Business',
     'lat': 43.77276955980532,
     'lng': -79.49880289347168,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.77276955980532,
       'lng': -79.49880289347168}],
     'distance': 292,
     'postalCode': 'M3J 3K1',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['4700 Keele St. (at Schulich School of Business)',
      'Toronto ON M3J 3K1',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d1e0931735',
      'name': 'Coffee Shop',
      'pluralName': 'Coffee Shops',
      'shortName': 'Coffee Shop',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
       'suffix': '.png'},
      'primary': True}],
    'refer

In [54]:
#Get the relevant part of JSON and transform it into a pandas dataframe assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress
0,4c878e539c808cfa582e151e,Timothy's World Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1607585058,False,4700 Keele St.,at Schulich School of Business,43.772770,-79.498803,"[{'label': 'display', 'lat': 43.77276955980532...",292,M3J 3K1,CA,Toronto,ON,Canada,[4700 Keele St. (at Schulich School of Busines...
1,4b0c25b5f964a520ce3723e3,Absinthe Pub and Coffee Shop,"[{'id': '4bf58dd8d48988d11b941735', 'name': 'P...",v-1607585058,False,4700 Keele St,Keele and Steeles,43.776412,-79.501995,"[{'label': 'display', 'lat': 43.77641202695796...",335,NaN,CA,Toronto,ON,Canada,"[4700 Keele St (Keele and Steeles), Toronto ON..."


In [55]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Timothy's World Coffee,Coffee Shop,4700 Keele St.,at Schulich School of Business,43.772770,-79.498803,"[{'label': 'display', 'lat': 43.77276955980532...",292,M3J 3K1,CA,Toronto,ON,Canada,[4700 Keele St. (at Schulich School of Busines...,4c878e539c808cfa582e151e
1,Absinthe Pub and Coffee Shop,Pub,4700 Keele St,Keele and Steeles,43.776412,-79.501995,"[{'label': 'display', 'lat': 43.77641202695796...",335,NaN,CA,Toronto,ON,Canada,"[4700 Keele St (Keele and Steeles), Toronto ON...",4b0c25b5f964a520ce3723e3


In [56]:
venues_map = folium.Map(location=[york_university_latitude,york_university_longitude,], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Toronto District School Board
folium.CircleMarker(
    [york_university_latitude, york_university_longitude],
    radius=10,
    color='red',
    popup='York University',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the coffee shops as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.name):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

* **There are 2 coffee shops that is really close to York University with distance around 300m** : Timothy's World Coffee & Absinthe Pub and Coffee Shop
* The competition to acquire the students as our main targets may exist, but it is not that bad, at least it is not as crowded as the first North York cluster.

2. Ontario Industrial and Finishing Skills Centre in York University Heights, 27

In [57]:
#Using the operator .loc[] to select data and see the longitude & latitude 
ontarioskills_york = df.loc["Ontario Industrial and Finishing Skills Centre"]
ontarioskills_york

ORGANIZATION_ADDRESS         130 Toro Rd, Unit C, Toronto, ON     M3J 3M9
NEIGHBOURHOOD                                 York University Heights, 27
DESCRIPTION_SERVICE     Apprenticeship programs for painter decorators...
LONGITUDE                                                        -79.4832
LATITUDE                                                          43.7619
ADDRESS_FULL                                                  130 Toro Rd
MUNICIPALITY                                                   North York
POSTAL_CODE                                                       M3J 3M9
Name: Ontario Industrial and Finishing Skills Centre, dtype: object

In [58]:
ontarioskills_york_latitude = ontarioskills_york['LATITUDE']
ontarioskills_york_longitude = ontarioskills_york['LONGITUDE']
print('Latitude and longitude values of Toronto Catholic District School Board are {}, {}.'.format(liuna_downsview_latitude, 
                                                               liuna_downsview_longitude))

Latitude and longitude values of Toronto Catholic District School Board are 43.7242755262, -79.4915887618.


In [59]:
#Explore coffee shops around LIUNA Local 183 Training Centre with radius 1Km
radius = 1000
search_query = 'Coffee'
# create URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    ontarioskills_york_latitude, 
    ontarioskills_york_longitude,
    VERSION,
    search_query,
    radius,
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/search?client_id=3KZIEIWOYFJZIM3SOLIV3EABZP5XOTOVVQVBJJF2CPH4ECOE&client_secret=PJZ5VDM0A2I1JDQDDVXJ2XIGHXPLH4VVIPC0EREVOCNQSE4D&ll=43.7619063057,-79.4832149226&v=20180604&query=Coffee&radius=1000&limit=30'

In [60]:
#Send the GET Request and examine the results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fd1cd22b945750a9b1db67e'},
 'response': {'venues': [{'id': '4e0a9e3752b1387fe6efee03',
    'name': 'Gold Star Coffee',
    'location': {'address': '4025 Chesswood Drive',
     'lat': 43.75913158843968,
     'lng': -79.47470197127578,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.75913158843968,
       'lng': -79.47470197127578}],
     'distance': 750,
     'cc': 'CA',
     'city': 'North York',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['4025 Chesswood Drive', 'North York ON', 'Canada']},
    'categories': [{'id': '4eb1bea83b7b6f98df247e06',
      'name': 'Factory',
      'pluralName': 'Factories',
      'shortName': 'Factory',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/factory_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1607585058',
    'hasPerk': False},
   {'id': '4c5edc7afff99c74ad1153d3',
    'name': 'Coffee Time',
    'location': {'addre

In [61]:
#Get the relevant part of JSON and transform it into a pandas dataframe assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.postalCode
0,4e0a9e3752b1387fe6efee03,Gold Star Coffee,"[{'id': '4eb1bea83b7b6f98df247e06', 'name': 'F...",v-1607585058,False,4025 Chesswood Drive,43.759132,-79.474702,"[{'label': 'display', 'lat': 43.75913158843968...",750,CA,North York,ON,Canada,"[4025 Chesswood Drive, North York ON, Canada]",NaN,NaN
1,4c5edc7afff99c74ad1153d3,Coffee Time,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1607585058,False,701 Chesswood Dr,43.755950,-79.474088,"[{'label': 'display', 'lat': 43.75594982233461...",989,CA,Toronto,ON,Canada,"[701 Chesswood Dr (Sheppard Ave), Toronto ON, ...",Sheppard Ave,NaN
2,530b99c4498e9d01f3650875,Cultured Coffee Bean,"[{'id': '4bf58dd8d48988d1f9941735', 'name': 'F...",v-1607585058,False,3946 Chesswood Dr,43.755301,-79.478178,"[{'label': 'display', 'lat': 43.755301, 'lng':...",839,CA,North York,ON,Canada,"[3946 Chesswood Dr, North York ON M3J 2W6, Can...",NaN,M3J 2W6


In [62]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,crossStreet,postalCode,id
0,Gold Star Coffee,Factory,4025 Chesswood Drive,43.759132,-79.474702,"[{'label': 'display', 'lat': 43.75913158843968...",750,CA,North York,ON,Canada,"[4025 Chesswood Drive, North York ON, Canada]",NaN,NaN,4e0a9e3752b1387fe6efee03
1,Coffee Time,Coffee Shop,701 Chesswood Dr,43.755950,-79.474088,"[{'label': 'display', 'lat': 43.75594982233461...",989,CA,Toronto,ON,Canada,"[701 Chesswood Dr (Sheppard Ave), Toronto ON, ...",Sheppard Ave,NaN,4c5edc7afff99c74ad1153d3
2,Cultured Coffee Bean,Food & Drink Shop,3946 Chesswood Dr,43.755301,-79.478178,"[{'label': 'display', 'lat': 43.755301, 'lng':...",839,CA,North York,ON,Canada,"[3946 Chesswood Dr, North York ON M3J 2W6, Can...",NaN,M3J 2W6,530b99c4498e9d01f3650875


In [63]:
venues_map = folium.Map(location=[ontarioskills_york_latitude,ontarioskills_york_longitude,], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Toronto District School Board
folium.CircleMarker(
    [ontarioskills_york_latitude, ontarioskills_york_longitude],
    radius=10,
    color='red',
    popup='Ontario Industrial and Finishing Skills Centre',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the coffee shops as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.name):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

There are 3 coffee shops in Ontario Industrial and Finishing Skills Centre's neigbourhood : Coffee Time (900m), Cultured Coffee Bean (800m) & Gold Star Coffee (750m).

***
<font size="3"><font color='blue'> There are 5 coffee shops in this street area, all of them are in the distance less than 1 Km from the learning centres. </font></font>
***

### 4C.2 Explore Learning Centres Area In Former Toronto City (4 th Street Cluster)

1. Ryerson University in Church-Yonge Corridor, 75

In [64]:
#Using the operator .loc[] to select data and see the longitude & latitude 
Ryerson_yonge = df.loc["Ryerson University"]
Ryerson_yonge

ORGANIZATION_ADDRESS             350 Victoria St, Toronto, ON     M5B 2K3
NEIGHBOURHOOD                                   Church-Yonge Corridor, 75
DESCRIPTION_SERVICE     Full and part time undergraduate and graduate ...
LONGITUDE                                                        -79.3805
LATITUDE                                                          43.6578
ADDRESS_FULL                                              350 Victoria St
MUNICIPALITY                                               former Toronto
POSTAL_CODE                                                       M5B 2K3
Name: Ryerson University, dtype: object

In [65]:
Ryerson_yonge_latitude = Ryerson_yonge['LATITUDE']
Ryerson_yonge_longitude = Ryerson_yonge['LONGITUDE']
print('Latitude and longitude values of Ryerson University are {}, {}.'.format(liuna_downsview_latitude, 
                                                               liuna_downsview_longitude))

Latitude and longitude values of Ryerson University are 43.7242755262, -79.4915887618.


In [66]:
#Explore coffee shops with radius 1Km
radius = 1000
search_query = 'Coffee'
# create URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    Ryerson_yonge_latitude, 
    Ryerson_yonge_longitude,
    VERSION,
    search_query,
    radius,
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/search?client_id=3KZIEIWOYFJZIM3SOLIV3EABZP5XOTOVVQVBJJF2CPH4ECOE&client_secret=PJZ5VDM0A2I1JDQDDVXJ2XIGHXPLH4VVIPC0EREVOCNQSE4D&ll=43.6577656355,-79.380479826&v=20180604&query=Coffee&radius=1000&limit=30'

In [67]:
#Send the GET Request and examine the results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fd1cd23e08ec61addd240dd'},
 'response': {'venues': [{'id': '4fb13c20e4b011e6f93513c0',
    'name': "Balzac's Coffee",
    'location': {'address': '122 Bond Street',
     'crossStreet': 'at Gould St.',
     'lat': 43.65785440672277,
     'lng': -79.37919981155157,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.65785440672277,
       'lng': -79.37919981155157}],
     'distance': 103,
     'postalCode': 'M5B 1X8',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['122 Bond Street (at Gould St.)',
      'Toronto ON M5B 1X8',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d1e0931735',
      'name': 'Coffee Shop',
      'pluralName': 'Coffee Shops',
      'shortName': 'Coffee Shop',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1607585059',
    'hasPerk': Fa

In [68]:
#Get the relevant part of JSON and transform it into a pandas dataframe assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood,venuePage.id
0,4fb13c20e4b011e6f93513c0,Balzac's Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1607585059,False,122 Bond Street,at Gould St.,43.657854,-79.379200,"[{'label': 'display', 'lat': 43.65785440672277...",103,M5B 1X8,CA,Toronto,ON,Canada,"[122 Bond Street (at Gould St.), Toronto ON M5...",NaN,NaN
1,4ec514ec9911232436e364af,Timothy's World Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1607585059,False,Yonge,Dundas,43.656700,-79.379941,"[{'label': 'display', 'lat': 43.65669995833159...",126,M5B 2G9,CA,Toronto,ON,Canada,"[Yonge (Dundas), Toronto ON M5B 2G9, Canada]",NaN,NaN
2,59f784dd28122f14f9d5d63d,HotBlack Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1607585059,False,245 Queen Street West,at St Patrick St,43.650364,-79.388669,"[{'label': 'display', 'lat': 43.65036434800487...",1055,M5V 1Z4,CA,Toronto,ON,Canada,"[245 Queen Street West (at St Patrick St), Tor...",Entertainment District,463001529
3,4bce5e21cc8cd13a7359c4cf,Timothy's World Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1607585059,False,444 Yonge St,in College Park,43.660467,-79.384654,"[{'label': 'display', 'lat': 43.66046739684086...",451,M5B 2H4,CA,Toronto,ON,Canada,"[444 Yonge St (in College Park), Toronto ON M5...",NaN,NaN
4,4b0aaa8ef964a520272623e3,Timothy's World Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1607585059,False,"483 Bay St, Bell Trinity Square",Bell Trinity Square,43.653436,-79.382314,"[{'label': 'display', 'lat': 43.653436, 'lng':...",504,M5G 2C9,CA,Toronto,ON,Canada,"[483 Bay St, Bell Trinity Square (Bell Trinity...",NaN,NaN


In [69]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Balzac's Coffee,Coffee Shop,122 Bond Street,at Gould St.,43.657854,-79.379200,"[{'label': 'display', 'lat': 43.65785440672277...",103,M5B 1X8,CA,Toronto,ON,Canada,"[122 Bond Street (at Gould St.), Toronto ON M5...",NaN,4fb13c20e4b011e6f93513c0
1,Timothy's World Coffee,Coffee Shop,Yonge,Dundas,43.656700,-79.379941,"[{'label': 'display', 'lat': 43.65669995833159...",126,M5B 2G9,CA,Toronto,ON,Canada,"[Yonge (Dundas), Toronto ON M5B 2G9, Canada]",NaN,4ec514ec9911232436e364af
2,HotBlack Coffee,Coffee Shop,245 Queen Street West,at St Patrick St,43.650364,-79.388669,"[{'label': 'display', 'lat': 43.65036434800487...",1055,M5V 1Z4,CA,Toronto,ON,Canada,"[245 Queen Street West (at St Patrick St), Tor...",Entertainment District,59f784dd28122f14f9d5d63d
3,Timothy's World Coffee,Coffee Shop,444 Yonge St,in College Park,43.660467,-79.384654,"[{'label': 'display', 'lat': 43.66046739684086...",451,M5B 2H4,CA,Toronto,ON,Canada,"[444 Yonge St (in College Park), Toronto ON M5...",NaN,4bce5e21cc8cd13a7359c4cf
4,Timothy's World Coffee,Coffee Shop,"483 Bay St, Bell Trinity Square",Bell Trinity Square,43.653436,-79.382314,"[{'label': 'display', 'lat': 43.653436, 'lng':...",504,M5G 2C9,CA,Toronto,ON,Canada,"[483 Bay St, Bell Trinity Square (Bell Trinity...",NaN,4b0aaa8ef964a520272623e3
5,Timothy's World Coffee,Coffee Shop,427 University Avenue,NaN,43.654053,-79.388090,"[{'label': 'display', 'lat': 43.65405317976302...",739,NaN,CA,Toronto,ON,Canada,"[427 University Avenue, Toronto ON, Canada]",NaN,4b44fc77f964a520cc0026e3
6,Timothy's World Coffee,Coffee Shop,401 Bay St.,at Richmond St. W,43.652135,-79.381172,"[{'label': 'display', 'lat': 43.65213455850074...",629,M5H 2Y4,CA,Toronto,ON,Canada,"[401 Bay St. (at Richmond St. W), Toronto ON M...",NaN,4baa9f6cf964a520817a3ae3
7,Timothy's World Coffee,Coffee Shop,801 Bay St,at College St,43.660714,-79.385491,"[{'label': 'display', 'lat': 43.66071353922905...",520,M5S 1Y9,CA,Toronto,ON,Canada,"[801 Bay St (at College St), Toronto ON M5S 1Y...",NaN,4b156e98f964a520cbac23e3
8,Timothy's World Coffee,Coffee Shop,425 University Ave,Dundas,43.654270,-79.387448,"[{'label': 'display', 'lat': 43.65427, 'lng': ...",682,M5G 1T6,CA,Toronto,ON,Canada,"[425 University Ave (Dundas), Toronto ON M5G 1...",NaN,53e8acc4498ee294fb100183
9,Second Cup Coffee Co. featuring Pinkberry Froz...,Café,"600 University Avenue, Room #202",NaN,43.657473,-79.390637,"[{'label': 'display', 'lat': 43.657473, 'lng':...",818,M5G 1X5,CA,Toronto,ON,Canada,"[600 University Avenue, Room #202, Toronto ON ...",NaN,5c86b682da2e00002cf95781


In [70]:
venues_map = folium.Map(location=[Ryerson_yonge_latitude,Ryerson_yonge_longitude,], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Toronto District School Board
folium.CircleMarker(
    [Ryerson_yonge_latitude, Ryerson_yonge_longitude],
    radius=10,
    color='red',
    popup='Ryerson University',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the coffee shops as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.name):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

**This area is crowded with coffee shops! There are around 30 coffee shops** in Ryerson University's area, with :
<br>a. 12 coffee shops with < 500m distance
<br>b. 20 coffee shops located between 500m - 1000m

**The competition in this area is very high.** No wonder, since it is a University area.

2. University of Toronto in University, 79

In [71]:
#Using the operator .loc[] to select data and see the longitude & latitude 
toronto_university = df.loc["University of Toronto"]
toronto_university 

ORGANIZATION_ADDRESS       27 King's College Cir, Toronto, ON     M5S 1A1
NEIGHBOURHOOD                                              University, 79
DESCRIPTION_SERVICE     Full and part time undergraduate and graduate ...
LONGITUDE                                                        -79.3961
LATITUDE                                                          43.6609
ADDRESS_FULL                                       27 King's College Crcl
MUNICIPALITY                                               former Toronto
POSTAL_CODE                                                       M5S 1A1
Name: University of Toronto, dtype: object

In [72]:
toronto_university_latitude = toronto_university['LATITUDE']
toronto_university_longitude = toronto_university['LONGITUDE']
print('Latitude and longitude values of Toronto University are {}, {}.'.format(toronto_university_latitude, 
                                                               toronto_university_longitude))

Latitude and longitude values of Toronto University are 43.6609455297, -79.3960895108.


In [73]:
#Explore coffee shops around LIUNA Local 183 Training Centre with radius 1Km
radius = 1000
search_query = 'Coffee'
# create URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    toronto_university_latitude, 
    toronto_university_longitude,
    VERSION,
    search_query,
    radius,
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/search?client_id=3KZIEIWOYFJZIM3SOLIV3EABZP5XOTOVVQVBJJF2CPH4ECOE&client_secret=PJZ5VDM0A2I1JDQDDVXJ2XIGHXPLH4VVIPC0EREVOCNQSE4D&ll=43.6609455297,-79.3960895108&v=20180604&query=Coffee&radius=1000&limit=30'

In [74]:
#Get the relevant part of JSON and transform it into a pandas dataframe assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood,venuePage.id
0,4fb13c20e4b011e6f93513c0,Balzac's Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1607585059,False,122 Bond Street,at Gould St.,43.657854,-79.379200,"[{'label': 'display', 'lat': 43.65785440672277...",103,M5B 1X8,CA,Toronto,ON,Canada,"[122 Bond Street (at Gould St.), Toronto ON M5...",NaN,NaN
1,4ec514ec9911232436e364af,Timothy's World Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1607585059,False,Yonge,Dundas,43.656700,-79.379941,"[{'label': 'display', 'lat': 43.65669995833159...",126,M5B 2G9,CA,Toronto,ON,Canada,"[Yonge (Dundas), Toronto ON M5B 2G9, Canada]",NaN,NaN
2,59f784dd28122f14f9d5d63d,HotBlack Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1607585059,False,245 Queen Street West,at St Patrick St,43.650364,-79.388669,"[{'label': 'display', 'lat': 43.65036434800487...",1055,M5V 1Z4,CA,Toronto,ON,Canada,"[245 Queen Street West (at St Patrick St), Tor...",Entertainment District,463001529
3,4bce5e21cc8cd13a7359c4cf,Timothy's World Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1607585059,False,444 Yonge St,in College Park,43.660467,-79.384654,"[{'label': 'display', 'lat': 43.66046739684086...",451,M5B 2H4,CA,Toronto,ON,Canada,"[444 Yonge St (in College Park), Toronto ON M5...",NaN,NaN
4,4b0aaa8ef964a520272623e3,Timothy's World Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1607585059,False,"483 Bay St, Bell Trinity Square",Bell Trinity Square,43.653436,-79.382314,"[{'label': 'display', 'lat': 43.653436, 'lng':...",504,M5G 2C9,CA,Toronto,ON,Canada,"[483 Bay St, Bell Trinity Square (Bell Trinity...",NaN,NaN


In [75]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Balzac's Coffee,Coffee Shop,122 Bond Street,at Gould St.,43.657854,-79.379200,"[{'label': 'display', 'lat': 43.65785440672277...",103,M5B 1X8,CA,Toronto,ON,Canada,"[122 Bond Street (at Gould St.), Toronto ON M5...",NaN,4fb13c20e4b011e6f93513c0
1,Timothy's World Coffee,Coffee Shop,Yonge,Dundas,43.656700,-79.379941,"[{'label': 'display', 'lat': 43.65669995833159...",126,M5B 2G9,CA,Toronto,ON,Canada,"[Yonge (Dundas), Toronto ON M5B 2G9, Canada]",NaN,4ec514ec9911232436e364af
2,HotBlack Coffee,Coffee Shop,245 Queen Street West,at St Patrick St,43.650364,-79.388669,"[{'label': 'display', 'lat': 43.65036434800487...",1055,M5V 1Z4,CA,Toronto,ON,Canada,"[245 Queen Street West (at St Patrick St), Tor...",Entertainment District,59f784dd28122f14f9d5d63d
3,Timothy's World Coffee,Coffee Shop,444 Yonge St,in College Park,43.660467,-79.384654,"[{'label': 'display', 'lat': 43.66046739684086...",451,M5B 2H4,CA,Toronto,ON,Canada,"[444 Yonge St (in College Park), Toronto ON M5...",NaN,4bce5e21cc8cd13a7359c4cf
4,Timothy's World Coffee,Coffee Shop,"483 Bay St, Bell Trinity Square",Bell Trinity Square,43.653436,-79.382314,"[{'label': 'display', 'lat': 43.653436, 'lng':...",504,M5G 2C9,CA,Toronto,ON,Canada,"[483 Bay St, Bell Trinity Square (Bell Trinity...",NaN,4b0aaa8ef964a520272623e3
5,Timothy's World Coffee,Coffee Shop,427 University Avenue,NaN,43.654053,-79.388090,"[{'label': 'display', 'lat': 43.65405317976302...",739,NaN,CA,Toronto,ON,Canada,"[427 University Avenue, Toronto ON, Canada]",NaN,4b44fc77f964a520cc0026e3
6,Timothy's World Coffee,Coffee Shop,401 Bay St.,at Richmond St. W,43.652135,-79.381172,"[{'label': 'display', 'lat': 43.65213455850074...",629,M5H 2Y4,CA,Toronto,ON,Canada,"[401 Bay St. (at Richmond St. W), Toronto ON M...",NaN,4baa9f6cf964a520817a3ae3
7,Timothy's World Coffee,Coffee Shop,801 Bay St,at College St,43.660714,-79.385491,"[{'label': 'display', 'lat': 43.66071353922905...",520,M5S 1Y9,CA,Toronto,ON,Canada,"[801 Bay St (at College St), Toronto ON M5S 1Y...",NaN,4b156e98f964a520cbac23e3
8,Timothy's World Coffee,Coffee Shop,425 University Ave,Dundas,43.654270,-79.387448,"[{'label': 'display', 'lat': 43.65427, 'lng': ...",682,M5G 1T6,CA,Toronto,ON,Canada,"[425 University Ave (Dundas), Toronto ON M5G 1...",NaN,53e8acc4498ee294fb100183
9,Second Cup Coffee Co. featuring Pinkberry Froz...,Café,"600 University Avenue, Room #202",NaN,43.657473,-79.390637,"[{'label': 'display', 'lat': 43.657473, 'lng':...",818,M5G 1X5,CA,Toronto,ON,Canada,"[600 University Avenue, Room #202, Toronto ON ...",NaN,5c86b682da2e00002cf95781


In [76]:
venues_map = folium.Map(location=[toronto_university_latitude,toronto_university_longitude,], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Toronto District School Board
folium.CircleMarker(
    [toronto_university_latitude, toronto_university_longitude],
    radius=10,
    color='red',
    popup='Toronto University',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the coffee shops as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.name):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

Just like Ryerson University, **this street area is also crowded with coffee shops!** There are around 30 coffee shops around University of Toronto. It seems that all of the 30 coffee shops are competing to get customers from this two universities.

***
<font size="3"><font color='blue'> This street cluster area is very crowded with coffee shops. There are 30 coffee shops in the range of 1 Km to serve those two universities </font></font>
***

<font size="4"> <strong> Let's write the summary of our first step analysis :</strong></font>

* There are two boroughs that have the most number of universities/ learning centres : North York and Former Toronto City
* In the two boroughs, there area 4 clusters of streets which have most universities/ learning centres nearby 
<ol><li>Cluster 1 (North York) : Lansing - Westgate, 38 (Toronto District School Board) & Willowdale East, 51 (Toronto Catholic District School Board)</li>
    <li>Cluster 2 (North York) : Downsview-Roading-CFB, 26 (LIUNA Local Training Centre) & Maple Leaf, 29 (Conseil Scolaire Viamonde)</li>
    <li>Cluster 3 (North York) : York University Heights, 27 (York University) & York University Heights, 27 (Ontario Industrial and Finishing Skills Centre)</li>
    <li>Cluster 4 (Former Toronto) : Church-Yonge Corridor, 75 (Ryerson University) & University, 79 (University of Toronto)</li>
    </ol>
* After exploring the street neighbourhood using Foursquare API, we found out that :
<ol><li> In Cluster 1, there are 10 coffee shops < 1.5 Km. It looks like these streets are already well-served with coffee shops.</li>
    <li> In Cluster 2, there is only 1 coffee shop with the distance > 1.2 Km. This area is still under-served and the competition level is very low. </li>
    <li>In Cluster 3, there are 5 coffee shops in this street area and all of them are in the distance less than 1 Km from the learning centres</li>
    <li> In Cluster 4, there are 30 coffee shops in the range of 1 Km to serve two universities. This street cluster area is very crowded with coffee shops.</ol>
* From our first step of analysis we can see that **Cluster 2 is the most potential area where we can build our new coffee shop** with low level of competition (low coffee shop density) and two learning centres exist in the area

### 4D. Explore More Detail To Get More Data 

### Learn more detail about the competitor 

From the previous analysis, we know that **Cluster 2 is the most potential area** where we can build our new coffee shop. There is **low competition** because there is only **1 coffee shop in the area : 'Coffee Time'**. Let's find out more information about 'Coffee Time'.

#### Get the rating of Coffee Time

In [77]:
# The code was removed by Watson Studio for sharing.

In [78]:
venue_id = '4d46aec2c3e5f04db91fb020' # ID of Coffee Time
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


Since there is no rating for the coffee shop, let's check if there is some tips shared by the customers

#### Get Coffee Time's tips

In [79]:
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fd1cd24abab3133131419b5'},
 'response': {'tips': {'count': 0, 'items': []}}}

It looks like there is also no tips shared by the visitor about Coffee Time.

#### Let's more details about 'Coffee Time' cafe

In [80]:
url = 'https://api.foursquare.com/v2/venues/{}/?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION,)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fd1cd24e3f54d3244091473'},
 'response': {'venue': {'id': '4d46aec2c3e5f04db91fb020',
   'name': 'Coffee Time',
   'contact': {},
   'location': {'lat': 43.72851501,
    'lng': -79.47792573,
    'labeledLatLngs': [{'label': 'display',
      'lat': 43.72851501,
      'lng': -79.47792573}],
    'cc': 'CA',
    'country': 'Canada',
    'formattedAddress': ['Canada']},
   'canonicalUrl': 'https://foursquare.com/v/coffee-time/4d46aec2c3e5f04db91fb020',
   'categories': [{'id': '4bf58dd8d48988d16d941735',
     'name': 'Café',
     'pluralName': 'Cafés',
     'shortName': 'Café',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
      'suffix': '.png'},
     'primary': True}],
   'verified': False,
   'stats': {'tipCount': 0},
   'price': {'tier': 1, 'message': 'Cheap', 'currency': '$'},
   'likes': {'count': 1,
    'groups': [{'type': 'others',
      'count': 1,
      'items': [{'isSanctioned': False,
        'firstName': 'Dulce

**We get some details about Coffee Time:**
* There is only 1 like for Coffee Time, but no tips or check_in_counts. We assume the visitors for this coffee shops is only a few.
* The coffee shop is 'Tier-1' venue that serves low price

### Explore Cluster_2 area 

In [81]:
# The code was removed by Watson Studio for sharing.

In [82]:
#We are curious about the popular spots around Cluster_2. 
#In order to explore the area, let's start by getting the latitude and longitude values of LIUNA Local Training Centre, North York
latitude = 43.7242755262
longitude = -79.4915887618

In [83]:
# Define URL
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=3KZIEIWOYFJZIM3SOLIV3EABZP5XOTOVVQVBJJF2CPH4ECOE&client_secret=PJZ5VDM0A2I1JDQDDVXJ2XIGHXPLH4VVIPC0EREVOCNQSE4D&ll=43.7242755262,-79.4915887618&v=20180604&radius=5000&limit=100'

In [84]:
#Send GET request and examine results
import requests
results = requests.get(url).json()
'There are {} around LIUNA Local Training Centre, North York.'.format(len(results['response']['groups'][0]['items']))

'There are 100 around LIUNA Local Training Centre, North York.'

In [85]:
#Get relevant part of JSON
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4b8a7c74f964a520d26e32e3',
  'name': 'Rustic Bakery',
  'location': {'address': '318 Rustic Rd.',
   'crossStreet': 'at Culford Rd.',
   'lat': 43.715413998974796,
   'lng': -79.49030028640821,
   'labeledLatLngs': [{'label': 'display',
     'lat': 43.715413998974796,
     'lng': -79.49030028640821}],
   'distance': 991,
   'postalCode': 'M6L 1W5',
   'cc': 'CA',
   'city': 'North York',
   'state': 'ON',
   'country': 'Canada',
   'formattedAddress': ['318 Rustic Rd. (at Culford Rd.)',
    'North York ON M6L 1W5',
    'Canada']},
  'categories': [{'id': '4bf58dd8d48988d16a941735',
    'name': 'Bakery',
    'pluralName': 'Bakeries',
    'shortName': 'Bakery',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bakery_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 're

In [86]:
#Process JSON and convert it to a clean dataframe
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  from ipykernel import kernelapp as app


,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Rustic Bakery,Bakery,318 Rustic Rd.,at Culford Rd.,43.715414,-79.490300,"[{'label': 'display', 'lat': 43.71541399897479...",991,M6L 1W5,CA,North York,ON,Canada,"[318 Rustic Rd. (at Culford Rd.), North York O...",NaN,4b8a7c74f964a520d26e32e3
1,Hung Long,Vietnamese Restaurant,1122 Wilson Ave.,at Pleasant Home Blvd.,43.726524,-79.483659,"[{'label': 'display', 'lat': 43.72652393180744...",685,M3M 1G7,CA,Toronto,ON,Canada,"[1122 Wilson Ave. (at Pleasant Home Blvd.), To...",NaN,4d39fb9434ee37047185729b
2,Meatpoint,Steakhouse,NaN,NaN,43.727943,-79.478551,"[{'label': 'display', 'lat': 43.727943, 'lng':...",1125,M3K 1G7,CA,Toronto,ON,Canada,"[Toronto ON M3K 1G7, Canada]",NaN,5bb6b1cfacb37f002c0d70db
3,Get & Go Burrito,Burrito Place,1077 Wilson Ave,NaN,43.726534,-79.480398,"[{'label': 'display', 'lat': 43.72653359680858...",934,M3K 1G7,CA,Toronto,ON,Canada,"[1077 Wilson Ave, Toronto ON M3K 1G7, Canada]",NaN,52f68603498e956c97050a00
4,Euro Shawarma,Falafel Restaurant,2706 Keele St,NaN,43.726024,-79.481805,"[{'label': 'display', 'lat': 43.72602359134807...",810,NaN,CA,North York,ON,Canada,"[2706 Keele St, North York ON, Canada]",NaN,5109bb6ae4b0f4aaf4357792
5,True North Climbing,Climbing Gym,75 Carl Hall Rd,Downsview Park Sports Center,43.745507,-79.474332,"[{'label': 'display', 'lat': 43.74550748230518...",2740,M3K 2B9,CA,Toronto,ON,Canada,[75 Carl Hall Rd (Downsview Park Sports Center...,NaN,4b829a52f964a5204ada30e3
6,Crate & Barrel,Furniture / Home Store,3401 Dufferin Street,NaN,43.726584,-79.452661,"[{'label': 'display', 'lat': 43.72658365, 'lng...",3142,M6A 2T9,CA,Toronto,ON,Canada,"[3401 Dufferin Street, Toronto ON M6A 2T9, Can...",NaN,4b0edef7f964a520305c23e3
7,Bellwoods Brewery,Brewery,20 Hafis Rd,at Sheffield St,43.706577,-79.469328,"[{'label': 'display', 'lat': 43.70657721602504...",2662,M6J 2Z5,CA,Toronto,ON,Canada,"[20 Hafis Rd (at Sheffield St), Toronto ON M6J...",NaN,5855969c2b04f83e933e3188
8,LCBO,Liquor Store,675 Wilson Ave,btwn Dufferin St & Allen Rd,43.732717,-79.454717,"[{'label': 'display', 'lat': 43.73271669596289...",3111,M3K 1E3,CA,Toronto,ON,Canada,"[675 Wilson Ave (btwn Dufferin St & Allen Rd),...",NaN,4aeb6fdcf964a5200ec221e3
9,Yorkdale Shopping Centre,Shopping Mall,3401 Dufferin Street,NaN,43.725482,-79.452736,"[{'label': 'display', 'lat': 43.72548184266099...",3128,M6A 2T9,CA,Toronto,ON,Canada,"[3401 Dufferin Street, Toronto ON M6A 2T9, Can...",NaN,4ad4c062f964a52002f820e3


In [87]:
#Let's visualize these items on the map around our location
venues_map = folium.Map(location=[latitude, longitude], zoom_start=10) # generate map centred around LIUNA


# add Ecco as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='LIUNA',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

In [88]:
#find out how many unique palces can be curated from all the returned venues
print('There are {} uniques names.'.format(len(dataframe_filtered['name'].unique())))

There are 96 uniques names.


In [89]:
# selecting rows based on condition : take places with radius < 2.5 Km
rslt_df = dataframe_filtered.loc[dataframe_filtered['distance'] < 2500] 

In [90]:
#find out how many categories, to see what type of places are nearby
rslt_df['categories'].value_counts()

Vietnamese Restaurant      4
Airport                    1
Thrift / Vintage Store     1
Coffee Shop                1
Grocery Store              1
Burrito Place              1
Cosmetics Shop             1
Boutique                   1
Liquor Store               1
Turkish Restaurant         1
Steakhouse                 1
Bakery                     1
Pet Store                  1
Chinese Restaurant         1
Furniture / Home Store     1
Hockey Arena               1
Falafel Restaurant         1
New American Restaurant    1
Athletics & Sports         1
Name: categories, dtype: int64

**It looks like Cluster_2 is majorly surrounded by restaurants & stores, a busy area .The neighbourbood in Cluster_2 is strategically beneficial for our new coffee shop.**

Note : The data in 'dataframe filtered' table also shows that there is a coffee shop named 'Tim Horton' in this area. After further checking, 'Tim Horton' cafe is not a coffee shop, but a donut shop - similar to Dunkin Donuts. Therefore, it is not our direct competitors.

### 4E. Segmenting and Clustering 

We already know that although there is only 1 coffee shop in Cluster_2 (this area is still underserved with coffee shop), it is actually a busy area surrounded by restaurants and store (in distance < 2.5 Km). Next, we want to cluster the boroughs to learn the characteristics. 

But before that, let's find out about the trending venues nearby university/ learning centers in Toronto City.

#### Explore Trending Venues

In [91]:
address = 'North York, Toronto, ON, Canada'

geolocator = Nominatim(user_agent="northyork_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.7543263 -79.44911696639593


In [92]:
# The code was removed by Watson Studio for sharing.

{'meta': {'code': 200, 'requestId': '5fd1cd277d6cea4fd3d5cf00'},
 'response': {'venues': []}}

In [93]:
#Check if any venues are trending at this time
# depending on when you run the above code, you might get different venues since the venues with the highest foot traffic are fetched live
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [94]:
# display trending venues
trending_venues_df

'No trending venues are available at the moment!'

In [95]:
#Visualize trending venues
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


    # add Ecco as a red circle mark
    folium.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Ecco',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)

In [96]:
# display map
venues_map

'Cannot generate visual as no trending venues are available at the moment!'

We couldn't find any trending venues in Norh York, ON at the time the code was running, let's try to cluster the area based on the universities/ learning centres & the venues nearby.

#### Segmenting & Clustering Neighbourhood Around The Universities/ Learning Centres

In [97]:
# df display 
df

,ORGANIZATION_ADDRESS,NEIGHBOURHOOD,DESCRIPTION_SERVICE,LONGITUDE,LATITUDE,ADDRESS_FULL,MUNICIPALITY,POSTAL_CODE
AGENCY_NAME,,,,,,,,
Joint Training and Apprenticeship Committee,"936 Warden Ave, Toronto, ON M1L 4C9","Wexford-Maryvale, 119",Apprenticeship programs in plumbing and steamf...,-79.290015,43.731531,936 Warden Ave,Scarborough,M1L 4C9
Ontario Industrial and Finishing Skills Centre,"130 Toro Rd, Unit C, Toronto, ON M3J 3M9","York University Heights, 27",Apprenticeship programs for painter decorators...,-79.483215,43.761906,130 Toro Rd,North York,M3J 3M9
Toronto Ironworkers Local 721,"909 Kipling Ave, Toronto, ON M8Z 5H3","Islington-City Centre West, 14",Apprenticeship programs for ironworkers and ro...,-79.533007,43.637679,909 Kipling Ave,Etobicoke,M8Z 5H3
Toronto Catholic District School Board,"Catholic Education Centre, 80 Sheppard Ave E, ...","Willowdale East, 51",Full academic program in a Roman Catholic envi...,-79.408096,43.762844,80 Sheppard Ave E,North York,M2N 6E8
Academy of Computer and Employment Skills,"55 Eglinton Ave E, Ste 703, Toronto, ON M4...","Mount Pleasant West, 104",Registered private career college * vocational...,-79.396078,43.706907,55 Eglinton Ave E,former Toronto,M4P 1G8
Redemption Reintegration Services,"1460 Midland Ave, Toronto, ON M1P 3B9","Dorset Park, 126",Reintegration services for African/Caribbean y...,-79.265626,43.752388,1460 Midland Ave,Scarborough,M1P 3B9
Conseil scolaire Viamonde,"116 Cornelius Pkwy, Toronto, ON M6L 2K5","Maple Leaf, 29","Academic, commercial and technical day courses...",-79.475535,43.717828,116 Cornelius Pkwy,North York,M6L 2K5
Toronto District School Board,"Head Office, 5050 Yonge St, Toronto, ON M2...","Lansing-Westgate, 38","Academic, commercial and technical, day and ni...",-79.413419,43.767049,5050 Yonge St,North York,M2N 5N8
Humber College,"Humber North Campus, 205 Humber College Blvd, ...","West Humber-Clairville, 1","Full and part time diploma, certificate, appre...",-79.606840,43.729167,205 Humber College Blvd,Etobicoke,M9W 5L7


In [98]:
#df appears with index (pivotted), reset the index as new dataframe
df_reset = df.reset_index(drop=False)

#df.reset_index(drop=True, inplace=True) --> if don't want to re-assingn use this code, drop = True if want to delete the index, add = False if not delete the index

In [99]:
df_reset

,AGENCY_NAME,ORGANIZATION_ADDRESS,NEIGHBOURHOOD,DESCRIPTION_SERVICE,LONGITUDE,LATITUDE,ADDRESS_FULL,MUNICIPALITY,POSTAL_CODE
0,Joint Training and Apprenticeship Committee,"936 Warden Ave, Toronto, ON M1L 4C9","Wexford-Maryvale, 119",Apprenticeship programs in plumbing and steamf...,-79.290015,43.731531,936 Warden Ave,Scarborough,M1L 4C9
1,Ontario Industrial and Finishing Skills Centre,"130 Toro Rd, Unit C, Toronto, ON M3J 3M9","York University Heights, 27",Apprenticeship programs for painter decorators...,-79.483215,43.761906,130 Toro Rd,North York,M3J 3M9
2,Toronto Ironworkers Local 721,"909 Kipling Ave, Toronto, ON M8Z 5H3","Islington-City Centre West, 14",Apprenticeship programs for ironworkers and ro...,-79.533007,43.637679,909 Kipling Ave,Etobicoke,M8Z 5H3
3,Toronto Catholic District School Board,"Catholic Education Centre, 80 Sheppard Ave E, ...","Willowdale East, 51",Full academic program in a Roman Catholic envi...,-79.408096,43.762844,80 Sheppard Ave E,North York,M2N 6E8
4,Academy of Computer and Employment Skills,"55 Eglinton Ave E, Ste 703, Toronto, ON M4...","Mount Pleasant West, 104",Registered private career college * vocational...,-79.396078,43.706907,55 Eglinton Ave E,former Toronto,M4P 1G8
5,Redemption Reintegration Services,"1460 Midland Ave, Toronto, ON M1P 3B9","Dorset Park, 126",Reintegration services for African/Caribbean y...,-79.265626,43.752388,1460 Midland Ave,Scarborough,M1P 3B9
6,Conseil scolaire Viamonde,"116 Cornelius Pkwy, Toronto, ON M6L 2K5","Maple Leaf, 29","Academic, commercial and technical day courses...",-79.475535,43.717828,116 Cornelius Pkwy,North York,M6L 2K5
7,Toronto District School Board,"Head Office, 5050 Yonge St, Toronto, ON M2...","Lansing-Westgate, 38","Academic, commercial and technical, day and ni...",-79.413419,43.767049,5050 Yonge St,North York,M2N 5N8
8,Humber College,"Humber North Campus, 205 Humber College Blvd, ...","West Humber-Clairville, 1","Full and part time diploma, certificate, appre...",-79.606840,43.729167,205 Humber College Blvd,Etobicoke,M9W 5L7
9,Seneca College,"Newnham Campus, 1750 Finch Ave E, Toronto, ON ...","Don Valley Village, 47",Full and part time diploma and certificate pro...,-79.348988,43.795781,1750 Finch Ave E,North York,M2J 2X5


In [100]:
df_reset.columns.tolist() #list column name in the table

['AGENCY_NAME',
 'ORGANIZATION_ADDRESS',
 'NEIGHBOURHOOD',
 'DESCRIPTION_SERVICE',
 'LONGITUDE',
 'LATITUDE',
 'ADDRESS_FULL',
 'MUNICIPALITY',
 'POSTAL_CODE']

In [101]:
#create a function to repeat the same process to all the neighborhoods in the universities/ learning centres location
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['School Name', 
                  'School Name Latitude', 
                  'School Name Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [102]:
#write the code to run the above function on each learning centres and create a new dataframe called df_venues.
df_venues = getNearbyVenues(names=df_reset['AGENCY_NAME'],
                                   latitudes=df_reset['LATITUDE'],
                                   longitudes=df_reset['LONGITUDE']
                                  )

Joint Training and Apprenticeship Committee
Ontario Industrial and Finishing Skills Centre
Toronto Ironworkers Local 721
Toronto Catholic District School Board
Academy of Computer and Employment Skills
Redemption Reintegration Services
Conseil scolaire Viamonde
Toronto District School Board
Humber College
Seneca College
University of Toronto
York University
Ryerson University
LIUNA Local 183 Training Centre
Ambassador Program
Conseil scolaire de district catholique Centre-Sud


In [103]:
#check the size of the resulting dataframe
print(df_venues.shape)
df_venues.head()

(854, 7)


,School Name,School Name Latitude,School Name Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Joint Training and Apprenticeship Committee,43.731531,-79.290015,Adonis,43.729188,-79.290391,Grocery Store
1,Joint Training and Apprenticeship Committee,43.731531,-79.290015,LCBO,43.729261,-79.292144,Liquor Store
2,Joint Training and Apprenticeship Committee,43.731531,-79.290015,The Corner Bank Sports Bar & Grill,43.731399,-79.288317,Sports Bar
3,Joint Training and Apprenticeship Committee,43.731531,-79.290015,California Sandwiches,43.726186,-79.287688,Sandwich Place
4,Joint Training and Apprenticeship Committee,43.731531,-79.290015,PetSmart,43.729945,-79.282226,Pet Store


In [104]:
#Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(df_venues['Venue Category'].unique())))

There are 186 uniques categories.


In [105]:
#check how many venues were returned for each learning centres
df_venues.groupby('School Name').count()

,School Name Latitude,School Name Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
School Name,,,,,,
Academy of Computer and Employment Skills,61,61,61,61,61,61
Ambassador Program,100,100,100,100,100,100
Conseil scolaire Viamonde,19,19,19,19,19,19
Conseil scolaire de district catholique Centre-Sud,49,49,49,49,49,49
Humber College,21,21,21,21,21,21
Joint Training and Apprenticeship Committee,90,90,90,90,90,90
LIUNA Local 183 Training Centre,19,19,19,19,19,19
Ontario Industrial and Finishing Skills Centre,26,26,26,26,26,26
Redemption Reintegration Services,36,36,36,36,36,36


***
<ol>
<li>There area 854 venues around all of the learning centres ( with distance less than < 5 Km)
<li>There are 186 uniques categories</li>
<li>Compares to others, learning centres in <Strong>Cluster_2 is among the lowest with number of venues in 5Km distance</strong>. After doing this comparison, we know that although there are restaurants & stores in Cluster_2 but compares to other learning centres, it looks like it is not a crowded area.</li> <br> Learning centres with most venues are :</ol>
<ul><li> Ambassador Program (Former Toronto)</li>
<li>Ryerson University (Former Toronto)</li>
<li>University of Toronto	(Former Toronto)</li>
<li>Toronto Catholic District School Board (North York)</li>
<li>Joint Training and Apprenticeship Committee (Scarborough)</li></ul>

***

In [106]:
#Analyze Each Learning Centres
# one hot encoding
df_onehot = pd.get_dummies(df_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
df_onehot['School Name'] = df_venues['School Name'] 

# move neighborhood column to the first column
fixed_columns = [df_onehot.columns[-1]] + list(df_onehot.columns[:-1])
df_onehot = df_onehot[fixed_columns]

df_onehot.head()

,School Name,Accessories Store,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,Bakery,Bank,Bar,Baseball Field,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Cafeteria,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,College Theater,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Electronics Store,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food & Drink Shop,Food Stand,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Health Food Store,Historic Site,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Laser Tag,Latin American Restaurant,Leather Goods Store,Lingerie Store,Liquor Store,Lounge,Massage Studio,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Neighborhood,New American Restaurant,Noodle House,Office,Optical Shop,Organic Grocery,Paper / Office Supplies Store,Park,Persian Restaurant,Pet Store,Pharmacy,Pide Place,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Road,Rock Club,Salad Place,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoothie Shop,Snack Place,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Thrift / Vintage Store,Track,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Joint Training and Apprenticeship Committee,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Joint Training and Apprenticeship Committee,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Joint Training and Apprenticeship Committee,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Joint Training and Apprenticeship Committee,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [107]:
#examine the new dataframe size
df_onehot.shape

(854, 187)

In [108]:
#group rows by learning centres and by taking the mean of the frequency of occurrence of each category
df_grouped = df_onehot.groupby('School Name').mean().reset_index()
df_grouped

,School Name,Accessories Store,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,Bakery,Bank,Bar,Baseball Field,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Cafeteria,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,College Theater,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Electronics Store,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food & Drink Shop,Food Stand,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Health Food Store,Historic Site,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Laser Tag,Latin American Restaurant,Leather Goods Store,Lingerie Store,Liquor Store,Lounge,Massage Studio,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Neighborhood,New American Restaurant,Noodle House,Office,Optical Shop,Organic Grocery,Paper / Office Supplies Store,Park,Persian Restaurant,Pet Store,Pharmacy,Pide Place,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Road,Rock Club,Salad Place,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoothie Shop,Snack Place,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Thrift / Vintage Store,Track,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Academy of Computer and Employment Skills,0.000000,0.000000,0.00,0.00,0.016393,0.000000,0.00,0.00,0.000000,0.016393,0.000000,0.016393,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.032787,0.00,0.000000,0.000000,0.016393,0.016393,0.000000,0.000000,0.00,0.016393,0.000000,0.000000,0.000000,0.000000,0.049180,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.081967,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.032787,0.000000,0.032787,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.016393,0.000000,0.00,0.016393,0.000000,0.00,0.000000,0.000000,0.016393,0.000000,0.016393,0.00,0.00,0.000000,0.000000,0.000000,0.032787,0.016393,0.016393,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.016393,0.016393,0.000000,0.098361,0.016393,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016393,0.000000,0.00,0.00,0.016393,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.049180,0.016393,0.000000,0.000000,0.00,0.049180,0.000000,0.016393,0.000000,0.000000,0.00,0.016393,0.016393,0.00,0.000000,0.032787,0.000000,0.00,0.016393,0.000000,0.016393,0.000000,0.000000,0.000000,0.016393,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.016393,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.016393,0.000000,0

In [109]:
#confirm the new size
df_grouped.shape

(16, 187)

In [110]:
#print each learning centres along with the top 5 most common venues
num_top_venues = 5

for hood in df_grouped['School Name']:
    print("----"+hood+"----")
    temp = df_grouped[df_grouped['School Name'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Academy of Computer and Employment Skills----
                venue  freq
0  Italian Restaurant  0.10
1         Coffee Shop  0.08
2                Café  0.05
3         Pizza Place  0.05
4                Park  0.05


----Ambassador Program----
                   venue  freq
0                   Café  0.08
1  Vietnamese Restaurant  0.06
2                    Bar  0.06
3                 Bakery  0.05
4            Coffee Shop  0.05


----Conseil scolaire Viamonde----
                  venue  freq
0        Clothing Store  0.16
1           Coffee Shop  0.11
2             Pet Store  0.11
3                  Park  0.05
4  Gym / Fitness Center  0.05


----Conseil scolaire de district catholique Centre-Sud----
                       venue  freq
0          Korean Restaurant  0.12
1                       Café  0.10
2                Coffee Shop  0.10
3  Middle Eastern Restaurant  0.06
4        Japanese Restaurant  0.04


----Humber College----
                    venue  freq
0             Coffee Sh

In [111]:
#put that into a pandas dataframe
#first let's write a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [112]:
#create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['School Name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['School Name'] = df_grouped['School Name']

for ind in np.arange(df_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,School Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Academy of Computer and Employment Skills,Italian Restaurant,Coffee Shop,Park,Pizza Place,Café,Bookstore,Gym,Dessert Shop,Deli / Bodega,Restaurant
1,Ambassador Program,Café,Vietnamese Restaurant,Bar,Coffee Shop,Bakery,Park,American Restaurant,Fast Food Restaurant,Thai Restaurant,Grocery Store
2,Conseil scolaire Viamonde,Clothing Store,Pet Store,Coffee Shop,Park,Intersection,Fast Food Restaurant,Breakfast Spot,Boutique,Chinese Restaurant,Gift Shop
3,Conseil scolaire de district catholique Centre...,Korean Restaurant,Café,Coffee Shop,Middle Eastern Restaurant,Grocery Store,Japanese Restaurant,Sandwich Place,Pizza Place,Bank,Bubble Tea Shop
4,Humber College,Coffee Shop,Bus Station,Sandwich Place,Restaurant,Fast Food Restaurant,Electronics Store,Steakhouse,Chinese Restaurant,Garden,Thrift / Vintage Store


### Cluster Neighborhoods

#### Run k-means to cluster the neighborhood into 4 clusters.

In [113]:
# set number of clusters
kclusters = 4

df_grouped_clustering = df_grouped.drop('School Name', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 0, 0, 3, 2, 3, 0, 3, 0, 3], dtype=int32)

In [114]:
#display df_reset
df_reset.head()

,AGENCY_NAME,ORGANIZATION_ADDRESS,NEIGHBOURHOOD,DESCRIPTION_SERVICE,LONGITUDE,LATITUDE,ADDRESS_FULL,MUNICIPALITY,POSTAL_CODE
0,Joint Training and Apprenticeship Committee,"936 Warden Ave, Toronto, ON M1L 4C9","Wexford-Maryvale, 119",Apprenticeship programs in plumbing and steamf...,-79.290015,43.731531,936 Warden Ave,Scarborough,M1L 4C9
1,Ontario Industrial and Finishing Skills Centre,"130 Toro Rd, Unit C, Toronto, ON M3J 3M9","York University Heights, 27",Apprenticeship programs for painter decorators...,-79.483215,43.761906,130 Toro Rd,North York,M3J 3M9
2,Toronto Ironworkers Local 721,"909 Kipling Ave, Toronto, ON M8Z 5H3","Islington-City Centre West, 14",Apprenticeship programs for ironworkers and ro...,-79.533007,43.637679,909 Kipling Ave,Etobicoke,M8Z 5H3
3,Toronto Catholic District School Board,"Catholic Education Centre, 80 Sheppard Ave E, ...","Willowdale East, 51",Full academic program in a Roman Catholic envi...,-79.408096,43.762844,80 Sheppard Ave E,North York,M2N 6E8
4,Academy of Computer and Employment Skills,"55 Eglinton Ave E, Ste 703, Toronto, ON M4...","Mount Pleasant West, 104",Registered private career college * vocational...,-79.396078,43.706907,55 Eglinton Ave E,former Toronto,M4P 1G8


In [115]:
#rename 'AGENCY_NAME' in df_rest into 'School Name' so that it can be merged
df_reset.rename(columns = {'AGENCY_NAME':'School Name'}, inplace = True)
df_reset

,School Name,ORGANIZATION_ADDRESS,NEIGHBOURHOOD,DESCRIPTION_SERVICE,LONGITUDE,LATITUDE,ADDRESS_FULL,MUNICIPALITY,POSTAL_CODE
0,Joint Training and Apprenticeship Committee,"936 Warden Ave, Toronto, ON M1L 4C9","Wexford-Maryvale, 119",Apprenticeship programs in plumbing and steamf...,-79.290015,43.731531,936 Warden Ave,Scarborough,M1L 4C9
1,Ontario Industrial and Finishing Skills Centre,"130 Toro Rd, Unit C, Toronto, ON M3J 3M9","York University Heights, 27",Apprenticeship programs for painter decorators...,-79.483215,43.761906,130 Toro Rd,North York,M3J 3M9
2,Toronto Ironworkers Local 721,"909 Kipling Ave, Toronto, ON M8Z 5H3","Islington-City Centre West, 14",Apprenticeship programs for ironworkers and ro...,-79.533007,43.637679,909 Kipling Ave,Etobicoke,M8Z 5H3
3,Toronto Catholic District School Board,"Catholic Education Centre, 80 Sheppard Ave E, ...","Willowdale East, 51",Full academic program in a Roman Catholic envi...,-79.408096,43.762844,80 Sheppard Ave E,North York,M2N 6E8
4,Academy of Computer and Employment Skills,"55 Eglinton Ave E, Ste 703, Toronto, ON M4...","Mount Pleasant West, 104",Registered private career college * vocational...,-79.396078,43.706907,55 Eglinton Ave E,former Toronto,M4P 1G8
5,Redemption Reintegration Services,"1460 Midland Ave, Toronto, ON M1P 3B9","Dorset Park, 126",Reintegration services for African/Caribbean y...,-79.265626,43.752388,1460 Midland Ave,Scarborough,M1P 3B9
6,Conseil scolaire Viamonde,"116 Cornelius Pkwy, Toronto, ON M6L 2K5","Maple Leaf, 29","Academic, commercial and technical day courses...",-79.475535,43.717828,116 Cornelius Pkwy,North York,M6L 2K5
7,Toronto District School Board,"Head Office, 5050 Yonge St, Toronto, ON M2...","Lansing-Westgate, 38","Academic, commercial and technical, day and ni...",-79.413419,43.767049,5050 Yonge St,North York,M2N 5N8
8,Humber College,"Humber North Campus, 205 Humber College Blvd, ...","West Humber-Clairville, 1","Full and part time diploma, certificate, appre...",-79.606840,43.729167,205 Humber College Blvd,Etobicoke,M9W 5L7
9,Seneca College,"Newnham Campus, 1750 Finch Ave E, Toronto, ON ...","Don Valley Village, 47",Full and part time diploma and certificate pro...,-79.348988,43.795781,1750 Finch Ave E,North York,M2J 2X5


In [116]:
#create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

df_merged = df_reset

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
df_merged = df_merged.join(neighborhoods_venues_sorted.set_index('School Name'), on='School Name')

df_merged.head()

,School Name,ORGANIZATION_ADDRESS,NEIGHBOURHOOD,DESCRIPTION_SERVICE,LONGITUDE,LATITUDE,ADDRESS_FULL,MUNICIPALITY,POSTAL_CODE,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Joint Training and Apprenticeship Committee,"936 Warden Ave, Toronto, ON M1L 4C9","Wexford-Maryvale, 119",Apprenticeship programs in plumbing and steamf...,-79.290015,43.731531,936 Warden Ave,Scarborough,M1L 4C9,3,Clothing Store,Fast Food Restaurant,Coffee Shop,Grocery Store,Furniture / Home Store,Sandwich Place,Burrito Place,Department Store,Shopping Plaza,Burger Joint
1,Ontario Industrial and Finishing Skills Centre,"130 Toro Rd, Unit C, Toronto, ON M3J 3M9","York University Heights, 27",Apprenticeship programs for painter decorators...,-79.483215,43.761906,130 Toro Rd,North York,M3J 3M9,3,Coffee Shop,Furniture / Home Store,Skating Rink,Japanese Restaurant,Metro Station,Caribbean Restaurant,Massage Studio,Fast Food Restaurant,Road,Restaurant
2,Toronto Ironworkers Local 721,"909 Kipling Ave, Toronto, ON M8Z 5H3","Islington-City Centre West, 14",Apprenticeship programs for ironworkers and ro...,-79.533007,43.637679,909 Kipling Ave,Etobicoke,M8Z 5H3,0,Coffee Shop,Bakery,Grocery Store,Sushi Restaurant,Restaurant,Fast Food Restaurant,Pizza Place,Turkish Restaurant,Bank,Beer Store
3,Toronto Catholic District School Board,"Catholic Education Centre, 80 Sheppard Ave E, ...","Willowdale East, 51",Full academic program in a Roman Catholic envi...,-79.408096,43.762844,80 Sheppard Ave E,North York,M2N 6E8,3,Coffee Shop,Japanese Restaurant,Pizza Place,Bank,Sandwich Place,Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Ramen Restaurant,Pharmacy
4,Academy of Computer and Employment Skills,"55 Eglinton Ave E, Ste 703, Toronto, ON M4...","Mount Pleasant West, 104",Registered private career college * vocational...,-79.396078,43.706907,55 Eglinton Ave E,former Toronto,M4P 1G8,3,Italian Restaurant,Coffee Shop,Park,Pizza Place,Café,Bookstore,Gym,Dessert Shop,Deli / Bodega,Restaurant


In [117]:
#let's visualize the resulting clusters
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_merged['LATITUDE'], df_merged['LONGITUDE'], df_merged['School Name'], df_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

#### Cluster_0 : The 'Asian Restaurants Dominant' Cluster

In [119]:
df_merged.loc[df_merged['Cluster Labels'] == 0, df_merged.columns[[0] + list(range(5, df_merged.shape[1]))]]

,School Name,LATITUDE,ADDRESS_FULL,MUNICIPALITY,POSTAL_CODE,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Toronto Ironworkers Local 721,43.637679,909 Kipling Ave,Etobicoke,M8Z 5H3,0,Coffee Shop,Bakery,Grocery Store,Sushi Restaurant,Restaurant,Fast Food Restaurant,Pizza Place,Turkish Restaurant,Bank,Beer Store
5,Redemption Reintegration Services,43.752388,1460 Midland Ave,Scarborough,M1P 3B9,0,Chinese Restaurant,Indian Restaurant,Pharmacy,Restaurant,Fast Food Restaurant,Asian Restaurant,Coffee Shop,Sandwich Place,Electronics Store,Clothing Store
6,Conseil scolaire Viamonde,43.717828,116 Cornelius Pkwy,North York,M6L 2K5,0,Clothing Store,Pet Store,Coffee Shop,Park,Intersection,Fast Food Restaurant,Breakfast Spot,Boutique,Chinese Restaurant,Gift Shop
10,University of Toronto,43.660946,27 King's College Crcl,former Toronto,M5S 1A1,0,Café,Coffee Shop,Bakery,Vegetarian / Vegan Restaurant,Bar,Beer Bar,Mexican Restaurant,Thai Restaurant,Burger Joint,Dessert Shop
13,LIUNA Local 183 Training Centre,43.724276,1263 Wilson Ave,North York,M3M 3G2,0,Vietnamese Restaurant,Burrito Place,Pizza Place,Supermarket,Restaurant,Coffee Shop,Gas Station,Falafel Restaurant,Baseball Field,Intersection
14,Ambassador Program,43.666710,28 Simpson Ave,former Toronto,M4K 1A2,0,Café,Vietnamese Restaurant,Bar,Coffee Shop,Bakery,Park,American Restaurant,Fast Food Restaurant,Thai Restaurant,Grocery Store


#### Cluster_1 : The 'Lack of Restaurants' Cluster

In [120]:
df_merged.loc[df_merged['Cluster Labels'] == 1, df_merged.columns[[0] + list(range(5, df_merged.shape[1]))]]

,School Name,LATITUDE,ADDRESS_FULL,MUNICIPALITY,POSTAL_CODE,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Seneca College,43.795781,1750 Finch Ave E,North York,M2J 2X5,1,Coffee Shop,Park,Bank,Sandwich Place,Pizza Place,Grocery Store,Gas Station,Intersection,Gym,Pharmacy


#### Cluster_2 : The 'Coffee Shop & Fast Food' Cluster

In [121]:
df_merged.loc[df_merged['Cluster Labels'] == 2, df_merged.columns[[0] + list(range(5, df_merged.shape[1]))]]

,School Name,LATITUDE,ADDRESS_FULL,MUNICIPALITY,POSTAL_CODE,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Humber College,43.729167,205 Humber College Blvd,Etobicoke,M9W 5L7,2,Coffee Shop,Bus Station,Sandwich Place,Restaurant,Fast Food Restaurant,Electronics Store,Steakhouse,Chinese Restaurant,Garden,Thrift / Vintage Store
11,York University,43.773410,4700 Keele St,North York,M3J 1P3,2,Coffee Shop,Pharmacy,Japanese Restaurant,Cosmetics Shop,Pizza Place,Chinese Restaurant,Pub,Café,Restaurant,Middle Eastern Restaurant


#### Cluster_3 : The 'Coffee Shop & Stores' Cluster

In [122]:
df_merged.loc[df_merged['Cluster Labels'] == 3, df_merged.columns[[0] + list(range(5, df_merged.shape[1]))]]

,School Name,LATITUDE,ADDRESS_FULL,MUNICIPALITY,POSTAL_CODE,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Joint Training and Apprenticeship Committee,43.731531,936 Warden Ave,Scarborough,M1L 4C9,3,Clothing Store,Fast Food Restaurant,Coffee Shop,Grocery Store,Furniture / Home Store,Sandwich Place,Burrito Place,Department Store,Shopping Plaza,Burger Joint
1,Ontario Industrial and Finishing Skills Centre,43.761906,130 Toro Rd,North York,M3J 3M9,3,Coffee Shop,Furniture / Home Store,Skating Rink,Japanese Restaurant,Metro Station,Caribbean Restaurant,Massage Studio,Fast Food Restaurant,Road,Restaurant
3,Toronto Catholic District School Board,43.762844,80 Sheppard Ave E,North York,M2N 6E8,3,Coffee Shop,Japanese Restaurant,Pizza Place,Bank,Sandwich Place,Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Ramen Restaurant,Pharmacy
4,Academy of Computer and Employment Skills,43.706907,55 Eglinton Ave E,former Toronto,M4P 1G8,3,Italian Restaurant,Coffee Shop,Park,Pizza Place,Café,Bookstore,Gym,Dessert Shop,Deli / Bodega,Restaurant
7,Toronto District School Board,43.767049,5050 Yonge St,North York,M2N 5N8,3,Japanese Restaurant,Korean Restaurant,Grocery Store,Pizza Place,Café,Burger Joint,Ramen Restaurant,Sushi Restaurant,Coffee Shop,Hotel
12,Ryerson University,43.657766,350 Victoria St,former Toronto,M5B 2K3,3,Coffee Shop,Japanese Restaurant,Gastropub,Italian Restaurant,Theater,Café,Hotel,Bookstore,Diner,Restaurant
15,Conseil scolaire de district catholique Centre...,43.786624,110 Drewry Ave,North York,M2M 1C8,3,Korean Restaurant,Café,Coffee Shop,Middle Eastern Restaurant,Grocery Store,Japanese Restaurant,Sandwich Place,Pizza Place,Bank,Bubble Tea Shop


* After examining the clusters, we can see that our targetted cluster (with learning centres : LIUNA Local 183 Training Centre & Conseil scolaire Viamonde) is included in the 'Asian Restaurants Dominant Cluster'
* Ambassador Program & LIUNA Local 183 Training Centre, both have quite the same characteristics of neighbourhood, with Vietnamese Restaurant is the top common venue in the neighbourhood
* Since in our first step of analysis, we cannot find out valuable detail about 'Coffee Time' (our potential direct competitor in Cluster_2 above), let's find out about the trending coffee shops in Ambassador Program, to get some ideas what kind of coffee shop that we should build in the neighbourhood with the same type of characteristics

### 4F. Find Out What Kind Of Coffee Shop We Should Built In The Neighbourhood

To get an ideas of what kind of coffee shop that we should build in Cluster_2 (the neighbourhood of our targetted learning centres : LIUNA Local 183 Training Centre & Conseil scolaire Viamonde), let's do some research regarding the 'Ambassador Program' learning centre which neighbourhood's characteristics is quite similar to our targetted learning centres.

In [123]:
#Search for coffee shops near 'Ambassador Program'
search_query = 'Coffee'
radius = 2500
print(search_query + ' .... OK!')

Coffee .... OK!


In [124]:
#Define latitude & longitude of 'Ambassador Program'
latitude = 43.666710
longitude = -79.351437

In [125]:
#Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=3KZIEIWOYFJZIM3SOLIV3EABZP5XOTOVVQVBJJF2CPH4ECOE&client_secret=PJZ5VDM0A2I1JDQDDVXJ2XIGHXPLH4VVIPC0EREVOCNQSE4D&ll=43.66671,-79.351437&v=20180604&query=Coffee&radius=2500&limit=100'

In [126]:
#Send the GET Request and examine the results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fd1cd30e957890549a2f9c5'},
 'response': {'venues': [{'id': '4b4626fef964a5204a1826e3',
    'name': 'Rooster Coffee House',
    'location': {'address': '479 Broadview Ave',
     'crossStreet': 'at Riverdale Ave',
     'lat': 43.669177107744204,
     'lng': -79.35313352914292,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.669177107744204,
       'lng': -79.35313352914292}],
     'distance': 306,
     'postalCode': 'M4K 2N4',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['479 Broadview Ave (at Riverdale Ave)',
      'Toronto ON M4K 2N4',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d16d941735',
      'name': 'Café',
      'pluralName': 'Cafés',
      'shortName': 'Café',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1607585072',
    'hasPerk': False},
  

In [127]:
#Get relevant part of JSON and transform it into a pandas dataframe
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood
0,4b4626fef964a5204a1826e3,Rooster Coffee House,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1607585072,False,479 Broadview Ave,at Riverdale Ave,43.669177,-79.353134,"[{'label': 'display', 'lat': 43.66917710774420...",306,M4K 2N4,CA,Toronto,ON,Canada,"[479 Broadview Ave (at Riverdale Ave), Toronto...",NaN
1,4ad79243f964a5204c0c21e3,Jetfuel Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1607585072,False,519 Parliament St.,btwn Carlton & Winchester,43.665295,-79.368335,"[{'label': 'display', 'lat': 43.66529519392083...",1369,M4X 1P3,CA,Toronto,ON,Canada,[519 Parliament St. (btwn Carlton & Winchester...,Cabbagetown
2,4dc04e8881545e1cc7e1b868,Timothy's World Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1607585072,False,2619 Yonge Street,NaN,43.674635,-79.363691,"[{'label': 'display', 'lat': 43.67463508704345...",1323,NaN,CA,Toronto,ON,Canada,"[2619 Yonge Street, Toronto ON, Canada]",NaN
3,5787c18b498e6faa6ae1f44d,Sakawa Coffee,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1607585072,False,867 Danforth Ave,NaN,43.679906,-79.339807,"[{'label': 'display', 'lat': 43.679906, 'lng':...",1742,M4J 1L8,CA,Toronto,ON,Canada,"[867 Danforth Ave, Toronto ON M4J 1L8, Canada]",NaN
4,4c9124191adc370489942dd1,Coffee Lime & Deli,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1607585072,False,968 Pape Ave.,NaN,43.687541,-79.348440,"[{'label': 'display', 'lat': 43.68754121766083...",2331,NaN,CA,Toronto,ON,Canada,"[968 Pape Ave., Toronto ON, Canada]",NaN


In [128]:
#Define information of interest and filter dataframe
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Rooster Coffee House,Café,479 Broadview Ave,at Riverdale Ave,43.669177,-79.353134,"[{'label': 'display', 'lat': 43.66917710774420...",306,M4K 2N4,CA,Toronto,ON,Canada,"[479 Broadview Ave (at Riverdale Ave), Toronto...",NaN,4b4626fef964a5204a1826e3
1,Jetfuel Coffee,Coffee Shop,519 Parliament St.,btwn Carlton & Winchester,43.665295,-79.368335,"[{'label': 'display', 'lat': 43.66529519392083...",1369,M4X 1P3,CA,Toronto,ON,Canada,[519 Parliament St. (btwn Carlton & Winchester...,Cabbagetown,4ad79243f964a5204c0c21e3
2,Timothy's World Coffee,Coffee Shop,2619 Yonge Street,NaN,43.674635,-79.363691,"[{'label': 'display', 'lat': 43.67463508704345...",1323,NaN,CA,Toronto,ON,Canada,"[2619 Yonge Street, Toronto ON, Canada]",NaN,4dc04e8881545e1cc7e1b868
3,Sakawa Coffee,Café,867 Danforth Ave,NaN,43.679906,-79.339807,"[{'label': 'display', 'lat': 43.679906, 'lng':...",1742,M4J 1L8,CA,Toronto,ON,Canada,"[867 Danforth Ave, Toronto ON M4J 1L8, Canada]",NaN,5787c18b498e6faa6ae1f44d
4,Coffee Lime & Deli,Coffee Shop,968 Pape Ave.,NaN,43.687541,-79.348440,"[{'label': 'display', 'lat': 43.68754121766083...",2331,NaN,CA,Toronto,ON,Canada,"[968 Pape Ave., Toronto ON, Canada]",NaN,4c9124191adc370489942dd1
5,Arena Coffee Bar,Coffee Shop,15 Tank House Lane,NaN,43.650280,-79.358860,"[{'label': 'display', 'lat': 43.65028, 'lng': ...",1924,M5A 3C4,CA,Toronto,ON,Canada,"[15 Tank House Lane, Toronto ON M5A 3C4, Canada]",NaN,58bdc3a23ef0f629212f1f70
6,Rooster Coffee House,Coffee Shop,568 Jarvis St,At Charles St E,43.669654,-79.379871,"[{'label': 'display', 'lat': 43.66965378571954...",2312,M4Y 1N6,CA,Toronto,ON,Canada,"[568 Jarvis St (At Charles St E), Toronto ON M...",NaN,569e7814498e1a7f3e01bfe4
7,Timothy's World Coffee,Coffee Shop,NaN,NaN,43.651230,-79.368457,"[{'label': 'display', 'lat': 43.65123, 'lng': ...",2201,NaN,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",NaN,4c3de34c7d002d7fe460b018
8,Coffee And Clothing,Coffee Shop,348 Pape Ave,Gerrard St E,43.668582,-79.340708,"[{'label': 'display', 'lat': 43.668582, 'lng':...",888,M4M 2X1,CA,Toronto,ON,Canada,"[348 Pape Ave (Gerrard St E), Toronto ON M4M 2...",NaN,5bddf75f16ef67002cff889b
9,Green Rooster Coffee Shop,Coffee Shop,Broadview,NaN,43.669489,-79.353454,"[{'label': 'display', 'lat': 43.669489, 'lng':...",349,NaN,CA,Toronto,ON,Canada,"[Broadview, Toronto ON, Canada]",NaN,4ba15904f964a52002af37e3


In [129]:
#Let's visualize the Coffee Shops that are nearby
dataframe_filtered.name

0                Rooster Coffee House
1                      Jetfuel Coffee
2              Timothy's World Coffee
3                       Sakawa Coffee
4                  Coffee Lime & Deli
5                    Arena Coffee Bar
6                Rooster Coffee House
7              Timothy's World Coffee
8                 Coffee And Clothing
9           Green Rooster Coffee Shop
10                    Balzac's Coffee
11             Timothy's World Coffee
12                      Hailed Coffee
13     Coffee Mobile - Brand Partners
14                  Fahrenheit Coffee
15             Timothy's World Coffee
16                  Coffee Enterprise
17       Luba’s Coffee & Tea Boutique
18                    Balzac's Coffee
19        Sam James Coffee Bar (SJCB)
20             Timothy's World Coffee
21             Timothy's World Coffee
22                   Supernova Coffee
23                        Coffee time
24                        Coffee Time
25        Congo Coffee + Mikate House
26          

In [130]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Ambassador Program',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.name):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [131]:
#Let's explore the closest coffee shop -- Rooster Coffee House
venue_id = '4b4626fef964a5204a1826e3' # ID of Rooster Coffee House
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4b4626fef964a5204a1826e3?client_id=3KZIEIWOYFJZIM3SOLIV3EABZP5XOTOVVQVBJJF2CPH4ECOE&client_secret=PJZ5VDM0A2I1JDQDDVXJ2XIGHXPLH4VVIPC0EREVOCNQSE4D&v=20180604'

In [132]:
#Send GET request for result
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'popular', 'seasonalHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4b4626fef964a5204a1826e3',
 'name': 'Rooster Coffee House',
 'contact': {'phone': '4169951530',
  'formattedPhone': '(416) 995-1530',
  'twitter': 'roostercoffee',
  'facebook': '149293765106089',
  'facebookUsername': 'RoosterCoffeeHouse',
  'facebookName': 'Rooster Coffee House'},
 'location': {'address': '479 Broadview Ave',
  'crossStreet': 'at Riverdale Ave',
  'lat': 43.669177107744204,
  'lng': -79.35313352914292,
  'labeledLatLngs': [{'label': 'display',
    'lat': 43.669177107744204,
    'lng': -79.35313352914292}],
  'postalCode': 'M4K 2N4',
  'cc': 'CA',
  'city': 'Toronto',
  'state': 'ON',
  'country': 'Canada',
  'formattedAddress': ['479 Broadview Ave (at Riverdale Ave)',
   'Toronto ON M4K 2N4',
   'Canada']},
 'canonicalUrl': 'https://foursquare.com/v/rooster-coffee-house/4b4626fef964a5204a1826e3',
 'categories': [{'id': '4bf58dd8d48988d16d941735',
   'name': 'Café',
   'pluralName': 'Cafés',
   'shortName': 'Café',
   'icon': {'prefix': 'https://ss3.4sqi.net/i

In [133]:
#Get the venue's overall rating
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

8.9


That's a really good rating for Rooster Coffee House!

In [134]:
#Get the number of tips
result['response']['venue']['tips']['count']

79

In [135]:
#Create URL and send GET request. Make sure to set limit to get all tips
limit = 100# set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fd1cd312e172d2de0c9b5de'},
 'response': {'tips': {'count': 79,
   'items': [{'id': '5aba8babea1e444f2a5dbfc7',
     'createdAt': 1522174891,
     'text': 'Their lattes are AMAZING & have you tried their beownies or cinnamon buns!?! To DIE for! & did I mention their lemonade oolala! Oh and staff is amazing, all so friendly and have great taste in music!!',
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/5aba8babea1e444f2a5dbfc7',
     'photo': {'id': '5aba8bb0b040560b82c94eaf',
      'createdAt': 1522174896,
      'source': {'name': 'Foursquare for iOS',
       'url': 'https://foursquare.com/download/#/iphone'},
      'prefix': 'https://fastly.4sqi.net/img/general/',
      'suffix': '/481550711_A0hJ64tFIBrBWlPaB4cGCEbYOyAaop1P87tTOeOi_Pg.jpg',
      'width': 1440,
      'height': 1920,
      'visibility': 'public'},
     'photourl': 'https://fastly.4sqi.net/img/general/original/481550711_A0hJ64tFIBrBWlPaB4cGCEbYOyAaop1P87tTOeO

In [136]:
#Get tips and list of associated features
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'photo', 'photourl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user', 'authorInteractionType'])

In [137]:
#Format column width and display all tips
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.id']
tips_filtered = tips_df.reindex(columns = filtered_columns)

# display tips
tips_filtered.reindex()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  from ipykernel import kernelapp as app
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.id
0,"Their lattes are AMAZING & have you tried their beownies or cinnamon buns!?! To DIE for! & did I mention their lemonade oolala! Oh and staff is amazing, all so friendly and have great taste in music!!",2,0,5aba8babea1e444f2a5dbfc7,Saba,B,NaN


** Note : I am using a personal developer account, so I can access only 1 of the restaurant's tips, instead of all tips.

From the tips shared, we can say that **customers like Rooster Coffee House's latte, brownies & cinnamon buns, lemonade drink, friendly staff, and music.**

In [138]:
#Get information about Roaster Coffee House's menu
#Define the corresponding URL
VENUE_ID = '4b4626fef964a5204a1826e3'
limit = 100
url = 'https://api.foursquare.com/v2/venues/{}/menu?client_id={}&client_secret={}&v={}&limit={}'.format(VENUE_ID, CLIENT_ID, CLIENT_SECRET, VERSION, limit)
url

'https://api.foursquare.com/v2/venues/4b4626fef964a5204a1826e3/menu?client_id=3KZIEIWOYFJZIM3SOLIV3EABZP5XOTOVVQVBJJF2CPH4ECOE&client_secret=PJZ5VDM0A2I1JDQDDVXJ2XIGHXPLH4VVIPC0EREVOCNQSE4D&v=20180604&limit=100'

In [139]:
#Send the GET Request and examine the results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fd1cd31b8aa6d5abd7b94ba'},
 'response': {'menu': {'menus': {'count': 0}}}}

It looks like we cannot get the detail of Roaster Coffee House's menu from Foursquare API, let's try to search it from Google or from its website.From Zomato website (https://www.zomato.com/toronto/rooster-coffee-house-riverdale) we found out that Roaster Coffee House is a cafe that focus mainly in coffee. It sold Espresso, Macchiato, Capuccino, etc. with a very affordable price, much more affordable than Starbucks Coffee.

## 5. Results & Discussion <a name="results"></a>

From the analysis, we find out that North York is the best borough for the new coffee shop. If we targeted students as the main customer segment, then this borough is perfect since it has the highest number of universities/ learning centres compares to other areas.<br>

Using the geographical visualization, we can identify which streets that have dense universities/ learning centres. After exploring the area using the Foursquare API, we recognize that there are well-served areas, while the others are already over-served. Fortunately, there is an area that is still under-served : the Downsview-Roading-CFB, 26 (LIUNA Local Training Centre) & Maple Leaf, 29 (Conseil Scolaire Viamonde).<br>

We choose this area as the potential location to build the new coffee shop to avoid harsh competition as we are new player to the business. But unfortunately, this specific of area, although it is surrounded by restaurants and stores, is not as crowded as other areas.
We decided not to build the new coffee shop 'exactly' in this two learning centres. The optimal location would be building the coffee shop not only that is reachable from this two learning centres (because there is 1 coffee shops that serves this area), but also comfortably reachable from Toronto Toronto Catholic District School Board.<br>

After clustering the borough using K-Means Clustering algorithm, we find out that 'Ambassador Program' learning centre is located in a neighbourhood with the same characteristics with the targeted neighbourhood, where the 1st common places in the neighbourhood is Vietnamese restaurants, therefore we assume that the characteristics of the visitors of the neighbourhood is also similar.<br>

We choose Roaster Coffee House, a coffee shop that is located near Ambassador Program learning centre, as a benchmark. It has good rating and has abundance list of tips that is shared by visitors. After studying Roaster Coffee Shop House in more detail using Foursquare API & website searchings, we are able to gain insights on what kind of coffee shop should we build in the neighbourhood, such as : <br>

* the menu does not need a lot of varieties, and better focus on classics coffee (espresso, capuccino, americano, macchiato, latte) with brownies or buns added to the menu
* the price of the coffee better in the range of $2 - $5
* built unique atmosphere for the coffee shop, Roaster Coffee House uses music

## 6. Conclusions <a name="conclusions"></a>

With data science, we can do early market research and gain insights that are useful for business. In this battle of neighbourhood of finding the optimal location to build a new coffee shop, we have generated several options and found the best place that match our criteria : low level of competition, located in a busy street, and reachable for students as our main customers (located not far from learning centres).